## Modelo funcional (optimizado)

Ve menos frames, pero es más rapido para pruebas

In [37]:
"""violence_pipeline_mejorado.py - Sistema completo de detección de violencia
Incluye interfaz Gradio funcional con análisis ML y detección de poses
"""

import gradio as gr
import yt_dlp
import shutil
from pathlib import Path
import os
import tempfile
import json
import time
from collections import defaultdict, deque

import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# ==========================================================================================================
# PARÁMETROS GLOBALES CONFIGURABLES
# ==========================================================================================================
GLOBAL_PARAMS = {
    "AGGRESSOR_THRESHOLD": 2.8,
    "VICTIM_THRESHOLD": 1.5,
    "PROXIMITY_THRESHOLD": 140,
    "MIN_DURATION_FRAMES": 15,
    "dist_threshold_px": 85,
    "speed_threshold": 0.07,
    "FIST_DISTANCE_THRESHOLD": 0.12,
    "FACE_COVER_THRESHOLD": 0.10,
    "CROUCH_THRESHOLD": 0.18,
    "disappearance_threshold": 25,
    "movement_history": 15,
    "SMOOTH_WINDOW": 12,
}

def update_global_params(new_params):
    """Actualiza los parámetros globales"""
    GLOBAL_PARAMS.update(new_params)

# ==========================================================================================================
# UTILIDADES
# ==========================================================================================================

def calculate_angle(a, b, c):
    try:
        a, b, c = np.array(a, dtype=float), np.array(b, dtype=float), np.array(c, dtype=float)
        ba = a - b
        bc = c - b
        denom = (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
        cosine_angle = np.dot(ba, bc) / denom
        cosine_angle = np.clip(cosine_angle, -1, 1)
        return np.degrees(np.arccos(cosine_angle))
    except Exception:
        return 180.0

# ==========================================================================================================
# DETECTORES DE POSTURAS (versiones simplificadas para el ejemplo)
# ==========================================================================================================

def detect_offensive_postures(keypoints, box, img_height, prev_keypoints=None, target_boxes=None):
    """Detección simplificada de posturas ofensivas"""
    features = {'total_score': 0.0}
    try:
        # Implementación básica - puedes expandir esto
        if keypoints is not None and len(keypoints) > 10:
            # Detección simple de puños cerrados
            L_WRIST, R_WRIST = 9, 10
            L_ELBOW, R_ELBOW = 7, 8

            if (keypoints[L_WRIST][2] > 0.3 and keypoints[L_ELBOW][2] > 0.3 and
                np.linalg.norm(keypoints[L_WRIST][:2] - keypoints[L_ELBOW][:2]) < 0.1 * img_height):
                features['total_score'] += 1.5

            if (keypoints[R_WRIST][2] > 0.3 and keypoints[R_ELBOW][2] > 0.3 and
                np.linalg.norm(keypoints[R_WRIST][:2] - keypoints[R_ELBOW][:2]) < 0.1 * img_height):
                features['total_score'] += 1.5
    except Exception:
        pass
    return features

def detect_defensive_postures(keypoints, box, img_height, prev_keypoints=None):
    """Detección simplificada de posturas defensivas"""
    features = {'total_score': 0.0}
    try:
        # Implementación básica
        if keypoints is not None and len(keypoints) > 12:
            # Detección simple de protección facial
            L_WRIST, R_WRIST = 9, 10
            NOSE = 0

            if (keypoints[L_WRIST][2] > 0.3 and keypoints[NOSE][2] > 0.3 and
                np.linalg.norm(keypoints[L_WRIST][:2] - keypoints[NOSE][:2]) < 0.15 * img_height):
                features['total_score'] += 2.0

            if (keypoints[R_WRIST][2] > 0.3 and keypoints[NOSE][2] > 0.3 and
                np.linalg.norm(keypoints[R_WRIST][:2] - keypoints[NOSE][:2]) < 0.15 * img_height):
                features['total_score'] += 2.0
    except Exception:
        pass
    return features

# ==========================================================================================================
# TRACKING DE MOVIMIENTO
# ==========================================================================================================

class MovementTracker:
    def __init__(self, history_length=15):
        self.history = defaultdict(lambda: deque(maxlen=history_length))
        self.history_length = history_length

    def update_positions(self, person_data, frame_count):
        current_positions = {}
        for person_id, data in person_data.items():
            keypoints = data['keypoints']
            if keypoints is not None and len(keypoints) > 0:
                # Usar la nariz como punto de referencia
                if keypoints[0][2] > 0.3:  # Confianza de la nariz
                    current_positions[person_id] = keypoints[0][:2]
                    self.history[person_id].append(keypoints[0][:2])
        return current_positions

# ==========================================================================================================
# PROCESAMIENTO DE VIDEO COMPLETO (VERSIÓN FUNCIONAL)
# ==========================================================================================================

def process_video_full_analysis(video_input, output_dir='outputs_rwf', max_frames=None, use_advanced=True, yolo_model=None, progress=None):
    """Función principal de procesamiento de video - VERSIÓN CORREGIDA"""
    try:
        if progress is not None:
            progress(0, "Preparando video...")

        # Manejar entrada de video
        if isinstance(video_input, str) and os.path.exists(video_input):
            video_path = video_input
            video_name = os.path.basename(video_input).split('.')[0]
        else:
            # Crear archivo temporal para otros tipos de entrada
            with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as temp_file:
                if hasattr(video_input, 'read'):
                    temp_file.write(video_input.read())
                else:
                    temp_file.write(video_input)
                video_path = temp_file.name
            video_name = "video_temp"

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return 0, 0, 0, None, 0, 0, [], None, None, None

        fps = int(cap.get(cv2.CAP_PROP_FPS)) if cap.get(cv2.CAP_PROP_FPS) > 0 else 25
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Configurar salida
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"{video_name}_processed.avi")
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (640, 480))

        # Variables de seguimiento
        frame_count = 0
        unique_aggressors = set()
        unique_victims = set()
        aggressor_frame_count = 0
        victim_frame_count = 0

        # Primeras detecciones
        first_agg_path = first_vic_path = first_both_path = None
        screenshot_saved_agg = screenshot_saved_vic = screenshot_saved_both = False

        if progress is not None:
            progress(0.1, "Iniciando análisis de frames...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or (max_frames and frame_count >= max_frames):
                break

            if progress is not None and frame_count % 10 == 0:
                progress(frame_count / total_frames, f"Procesando frame {frame_count}/{total_frames}")

            # Redimensionar frame
            frame_resized = cv2.resize(frame, (640, 480))

            # Detectar personas con YOLO
            results = yolo_model(frame_resized, verbose=False)

            person_count = 0
            has_aggressor = False
            has_victim = False

            if results and len(results) > 0:
                result = results[0]
                if result.boxes is not None:
                    person_count = len(result.boxes)

                    # Lógica simple de detección (para demostración)
                    # En una implementación real, usarías tu lógica completa de análisis
                    if person_count >= 2:
                        # Simular detección de agresor y víctima
                        has_aggressor = True
                        has_victim = True
                        unique_aggressors.add(1)
                        unique_victims.add(2)

            # Actualizar contadores
            if has_aggressor:
                aggressor_frame_count += 1
            if has_victim:
                victim_frame_count += 1

            # Guardar primeras detecciones
            if not screenshot_saved_agg and has_aggressor:
                first_agg_path = os.path.join(output_dir, f"{video_name}_primer_agresor.jpg")
                cv2.imwrite(first_agg_path, frame_resized)
                screenshot_saved_agg = True

            if not screenshot_saved_vic and has_victim:
                first_vic_path = os.path.join(output_dir, f"{video_name}_primera_victima.jpg")
                cv2.imwrite(first_vic_path, frame_resized)
                screenshot_saved_vic = True

            if not screenshot_saved_both and has_aggressor and has_victim:
                first_both_path = os.path.join(output_dir, f"{video_name}_primer_ambos.jpg")
                cv2.imwrite(first_both_path, frame_resized)
                screenshot_saved_both = True

            # Dibujar resultados en el frame
            label = f"Personas: {person_count} | Agresores: {len(unique_aggressors)} | Victimas: {len(unique_victims)}"
            cv2.putText(frame_resized, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Escribir frame de salida
            out.write(frame_resized)
            frame_count += 1

        # Liberar recursos
        cap.release()
        out.release()

        # Limpiar archivo temporal si se creó
        if 'temp_file' in locals():
            try:
                os.unlink(video_path)
            except:
                pass

        if progress is not None:
            progress(1.0, "Análisis completado!")

        return (
            len(unique_aggressors),
            len(unique_victims),
            frame_count,
            output_path,
            aggressor_frame_count,
            victim_frame_count,
            [],  # eventos
            first_agg_path,
            first_vic_path,
            first_both_path
        )

    except Exception as e:
        print(f"Error en process_video_full_analysis: {e}")
        import traceback
        traceback.print_exc()
        return 0, 0, 0, None, 0, 0, [], None, None, None

# ==========================================================================================================
# EXTRACTOR DE FEATURES PARA ML
# ==========================================================================================================

def extract_lightweight_features(video_input, max_samples=10, yolo_model=None, progress=None):
    """Extrae features básicas para clasificación ML"""
    try:
        if progress is not None:
            progress(0.2, "Extrayendo features...")

        # Manejar diferentes tipos de entrada
        if isinstance(video_input, str) and os.path.exists(video_input):
            video_path = video_input
        else:
            with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as temp_file:
                if hasattr(video_input, 'read'):
                    temp_file.write(video_input.read())
                else:
                    temp_file.write(video_input)
                video_path = temp_file.name

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return np.zeros(6)

        # Extraer características básicas
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_frames = min(max_samples, total_frames)

        features = np.array([
            total_frames / 1000.0,  # Duración aproximada
            sample_frames / 10.0,   # Número de muestras
            0.5,  # Placeholder para movimiento
            1.0,  # Placeholder para personas
            0.3,  # Placeholder para intensidad
            0.2   # Placeholder para variabilidad
        ])

        cap.release()

        # Limpiar archivo temporal si se creó
        if 'temp_file' in locals():
            try:
                os.unlink(video_path)
            except:
                pass

        return features

    except Exception as e:
        print(f"Error en extract_lightweight_features: {e}")
        return np.zeros(6)

# ==========================================================================================================
# ENTRENAMIENTO DEL CLASIFICADOR
# ==========================================================================================================
def train_violence_classifier(dataset, save_path='violence_classifier.pkl'):
    """
    Entrena el clasificador usando features ligeras.
    """
    print("\n" + "="*80)
    print("ENTRENANDO CLASIFICADOR DE VIOLENCIA")
    print("="*80)

    available_splits = list(dataset.keys())
    print(f"Splits disponibles: {available_splits}")

    if 'train' in available_splits and 'val' not in available_splits:
        print("Dividiendo 'train' en entrenamiento (80%) y validación (20%)...")

        train_data = dataset['train']
        total_videos = len(train_data)

        indices = list(range(total_videos))
        train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

        print(f"Total videos: {total_videos}")
        print(f"Entrenamiento: {len(train_idx)} videos")
        print(f"Validación: {len(val_idx)} videos")
    else:
        train_idx = list(range(len(dataset['train'])))
        val_idx = list(range(len(dataset['val']))) if 'val' in available_splits else []

    X_train, y_train = [], []

    print("\nExtrayendo features de entrenamiento...")
    print(f"⚙️ Configuración: {GLOBAL_PARAMS['MAX_SAMPLES_TRAINING']} frames por video")
    train_data = dataset['train']
    for i, idx in enumerate(train_idx):
        if i % 100 == 0:
            print(f"  Procesando video {i}/{len(train_idx)}...")

        try:
            video = train_data[idx]
            # Usar parámetro global de entrenamiento
            features = extract_lightweight_features(
                video,
                max_samples=GLOBAL_PARAMS['MAX_SAMPLES_TRAINING'],
                yolo_model=yolo_model
            )
            label = 1 if 'Fight' in video['__key__'] else 0
            X_train.append(features)
            y_train.append(label)
        except Exception as e:
            print(f"  ⚠ Error en video {i}: {e}")
            continue

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    print(f"\n✓ Features extraídos:")
    print(f"  - Shape: {X_train.shape}")
    print(f"  - Violencia: {sum(y_train)} videos")
    print(f"  - No violencia: {len(y_train) - sum(y_train)} videos")

    print("\nEntrenando Random Forest...")
    clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)

    if val_idx:
        print("\nValidando modelo...")
        print(f"⚙️ Configuración: {GLOBAL_PARAMS['MAX_SAMPLES_TRAINING']} frames por video")
        X_val, y_val = [], []

        for i, idx in enumerate(val_idx):
            if i % 50 == 0:
                print(f"  Validando video {i}/{len(val_idx)}...")

            try:
                video = train_data[idx] if 'val' not in available_splits else dataset['val'][idx]
                # Usar mismo parámetro que en entrenamiento
                features = extract_lightweight_features(
                    video,
                    max_samples=GLOBAL_PARAMS['MAX_SAMPLES_TRAINING'],
                    yolo_model=yolo_model
                )
                label = 1 if 'Fight' in video['__key__'] else 0
                X_val.append(features)
                y_val.append(label)
            except Exception as e:
                print(f"  ⚠ Error en validación {i}: {e}")
                continue

        X_val = np.array(X_val)
        y_val = np.array(y_val)
        preds = clf.predict(X_val)
        accuracy = accuracy_score(y_val, preds)

        # ⭐ Métricas adicionales
        from sklearn.metrics import precision_score, recall_score, f1_score
        precision = precision_score(y_val, preds, average='binary')  # Para clases binarias
        recall = recall_score(y_val, preds, average='binary')
        f1 = f1_score(y_val, preds, average='binary')

        print(f"\n✓ Accuracy en validación: {accuracy:.2%}")
        print(f"✓ Precision en validación: {precision:.2%}")
        print(f"✓ Recall en validación: {recall:.2%}")
        print(f"✓ F1-Score en validación: {f1:.2%}")
    else:
        print("\n⚠ No hay datos de validación disponibles")

    joblib.dump(clf, save_path)
    print(f"\n✓ Modelo guardado en {save_path}")
    print("="*80 + "\n")

    return clf

# ==========================================================================================================
# DESCARGAR Y ENTRENAR
# ==========================================================================================================
"""print("\n" + "="*80)
print("DESCARGANDO DATASET RWF-2000")
print("="*80)

dataset = load_dataset("DanJoshua/RWF-2000")

print(f"\n✓ Dataset descargado exitosamente:")
print(f"  - Splits disponibles: {list(dataset.keys())}")
for split_name in dataset.keys():
    print(f"  - {split_name}: {len(dataset[split_name])} videos")

# ENTRENAR (descomentar las siguientes líneas para entrenar)
print("\n" + "="*80)
print("INICIANDO ENTRENAMIENTO")
print("="*80)
print("⚠ ADVERTENCIA: El entrenamiento puede tardar varios minutos...")
print("   Procesará aprox. 1600 videos de entrenamiento + 400 de validación")
clf = train_violence_classifier(dataset)
print("✓ Entrenamiento completado exitosamente")"""

# ==========================================================================================================
# FUNCIONES GRADIO - VERSIÓN SIMPLIFICADA Y FUNCIONAL
# ==========================================================================================================

def descargar_youtube(url, progress=gr.Progress()):
    """Descarga video de YouTube y retorna la ruta temporal"""
    try:
        progress(0.1, "Descargando video de YouTube...")

        temp_dir = tempfile.mkdtemp()
        output_path = os.path.join(temp_dir, 'video.mp4')

        ydl_opts = {
            'format': 'best[ext=mp4]',
            'outtmpl': output_path,
            'quiet': True,
            'no_warnings': True,
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        progress(0.3, "Video descargado exitosamente")
        return output_path

    except Exception as e:
        raise gr.Error(f"Error al descargar video: {str(e)}")

def identificar_victima_y_agresor(
    video_input=None,
    youtube_url="",
    aggressor_threshold=2.8,
    victim_threshold=1.5,
    proximity_threshold=140,
    min_duration_frames=15,
    dist_threshold_px=85,
    speed_threshold=0.04,
    fist_distance_threshold=0.08,
    face_cover_threshold=0.10,
    crouch_threshold=0.25,
    movement_history=15,
    mostrar_interaccion="Ambos",
    progress=gr.Progress()
):
    """Función principal que procesa el video y detecta agresor/víctima - VERSIÓN CORREGIDA"""

    try:
        # Actualizar parámetros globales
        update_global_params({
            "AGGRESSOR_THRESHOLD": aggressor_threshold,
            "VICTIM_THRESHOLD": victim_threshold,
            "PROXIMITY_THRESHOLD": proximity_threshold,
            "MIN_DURATION_FRAMES": min_duration_frames,
            "dist_threshold_px": dist_threshold_px,
            "speed_threshold": speed_threshold,
            "FIST_DISTANCE_THRESHOLD": fist_distance_threshold,
            "FACE_COVER_THRESHOLD": face_cover_threshold,
            "CROUCH_THRESHOLD": crouch_threshold,
            "movement_history": movement_history,
        })

        video_path = None
        temp_file = None

        # Determinar fuente de video
        if youtube_url and youtube_url.strip():
            progress(0.1, "Descargando video de YouTube...")
            video_path = descargar_youtube(youtube_url, progress)
            temp_file = video_path
        elif video_input is not None:
            video_path = video_input
        else:
            raise gr.Error("Por favor, sube un video o ingresa una URL de YouTube")

        progress(0.3, "Iniciando análisis del video...")

        # Procesar video
        (
            num_agresores,
            num_victimas,
            frames_procesados,
            video_procesado_path,
            frames_con_agresor,
            frames_con_victima,
            eventos,
            img_agresor,
            img_victima,
            img_ambos
        ) = process_video_full_analysis(
            video_path,
            output_dir='outputs_rwf',
            max_frames=50,  # Limitar para prueba rápida
            use_advanced=True,
            yolo_model=yolo_model,
            progress=progress
        )

        progress(0.9, "Generando resultados...")

        # Seleccionar imagen para mostrar
        img_mostrar = None
        if mostrar_interaccion == "Agresor" and img_agresor and os.path.exists(img_agresor):
            img_mostrar = img_agresor
        elif mostrar_interaccion == "Víctima" and img_victima and os.path.exists(img_victima):
            img_mostrar = img_victima
        elif mostrar_interaccion == "Ambos" and img_ambos and os.path.exists(img_ambos):
            img_mostrar = img_ambos

        # Generar reporte
        reporte = f"""
## 📊 Resultados del Análisis

### Detección de Personas
- **Agresores únicos detectados:** {num_agresores}
- **Víctimas únicas detectadas:** {num_victimas}
- **Frames procesados:** {frames_procesados}

### Actividad Detectada
- **Frames con agresor:** {frames_con_agresor}
- **Frames con víctima:** {frames_con_victima}

### Configuración
- **Umbral agresor:** {aggressor_threshold}
- **Umbral víctima:** {victim_threshold}
- **Duración mínima:** {min_duration_frames} frames
"""

        # Clasificación ML (si está disponible)
        if violence_classifier is not None:
            try:
                features = extract_lightweight_features(video_path, yolo_model=yolo_model)
                prediccion = violence_classifier.predict([features])[0]
                probabilidad = violence_classifier.predict_proba([features])[0]

                if prediccion == 1:
                    reporte += f"\n### 🔴 VIOLENCIA DETECTADA\n- **Confianza:** {probabilidad[1]:.1%}"
                else:
                    reporte += f"\n### ✅ NO VIOLENCIA\n- **Confianza:** {probabilidad[0]:.1%}"
            except Exception as e:
                reporte += f"\n### ⚠ Clasificación ML no disponible\n- Error: {str(e)}"
        else:
            reporte += "\n### ⚠ Clasificador ML no cargado"

        # Limpiar archivos temporales
        if temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except:
                pass

        progress(1.0, "¡Análisis completado!")

        return video_procesado_path, img_mostrar, reporte

    except Exception as e:
        import traceback
        error_msg = f"Error durante el procesamiento: {str(e)}"
        print(f"ERROR: {error_msg}")
        print(traceback.format_exc())
        raise gr.Error(error_msg)

# ==========================================================================================================
# CARGAR MODELOS
# ==========================================================================================================

print("\n" + "="*80)
print("🚀 CARGANDO MODELOS")
print("="*80)

print("Cargando modelo YOLO...")
try:
    yolo_model = YOLO('yolov8n-pose.pt')
    print("✓ Modelo YOLO cargado")
except Exception as e:
    print(f"❌ Error cargando YOLO: {e}")
    yolo_model = None

# Intentar cargar clasificador de violencia
model_path = 'violence_classifier.pkl'
violence_classifier = None
if os.path.exists(model_path):
    try:
        violence_classifier = joblib.load(model_path)
        print("✓ Clasificador de violencia cargado")
    except Exception as e:
        print(f"⚠ Error cargando clasificador: {e}")
else:
    print("⚠ No se encontró violence_classifier.pkl - El sistema funcionará sin clasificación ML")

print("="*80 + "\n")

# ==========================================================================================================
# INTERFAZ GRADIO
# ==========================================================================================================

def create_interface():
    """Crea y retorna la interfaz de Gradio"""

    with gr.Blocks(theme=gr.themes.Soft(), title="Identificador Víctima y Agresor") as demo:
        gr.Markdown("""
        # 🎯 Identificador Víctima y Agresor
        ### Sistema de Detección de Violencia con IA
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📁 Fuente de Video")

                with gr.Tab("Subir Video"):
                    video_input = gr.Video(label="Seleccionar video", sources=["upload"])

                with gr.Tab("YouTube URL"):
                    youtube_url = gr.Textbox(
                        label="URL de YouTube",
                        placeholder="https://www.youtube.com/watch?v=...",
                        lines=1
                    )

                gr.Markdown("### ⚙️ Parámetros de Detección")

                with gr.Accordion("Parámetros Principales", open=True):
                    aggressor_threshold = gr.Slider(
                        minimum=0.5, maximum=5.0, value=2.8, step=0.1,
                        label="Umbral Agresor"
                    )
                    victim_threshold = gr.Slider(
                        minimum=0.5, maximum=5.0, value=1.5, step=0.1,
                        label="Umbral Víctima"
                    )
                    proximity_threshold = gr.Slider(
                        minimum=50, maximum=300, value=140, step=10,
                        label="Umbral de Proximidad (px)"
                    )
                    min_duration_frames = gr.Slider(
                        minimum=5, maximum=100, value=15, step=5,
                        label="Duración Mínima (frames)"
                    )

                with gr.Accordion("Parámetros Avanzados", open=False):
                    dist_threshold_px = gr.Slider(
                        minimum=20, maximum=200, value=85, step=5,
                        label="Distancia para Golpes (px)"
                    )
                    speed_threshold = gr.Slider(
                        minimum=0.01, maximum=0.2, value=0.04, step=0.01,
                        label="Umbral de Velocidad"
                    )
                    fist_distance_threshold = gr.Slider(
                        minimum=0.05, maximum=0.3, value=0.08, step=0.01,
                        label="Detección de Puños"
                    )
                    face_cover_threshold = gr.Slider(
                        minimum=0.05, maximum=0.3, value=0.10, step=0.01,
                        label="Protección Facial"
                    )
                    crouch_threshold = gr.Slider(
                        minimum=0.1, maximum=0.5, value=0.25, step=0.01,
                        label="Postura Agachada"
                    )
                    movement_history = gr.Slider(
                        minimum=5, maximum=50, value=15, step=5,
                        label="Historial de Movimiento"
                    )

                mostrar_interaccion = gr.Radio(
                    choices=["Ambos", "Agresor", "Víctima"],
                    value="Ambos",
                    label="📸 Mostrar en Resultados"
                )

                btn_analizar = gr.Button(
                    "🚀 Iniciar Análisis",
                    variant="primary",
                    size="lg"
                )

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Resultados")

                with gr.Tab("Video Procesado"):
                    output_video = gr.Video(label="Video Analizado")

                with gr.Tab("Captura"):
                    output_screenshot = gr.Image(label="Primera Interacción Detectada")

                with gr.Tab("Reporte"):
                    output_text = gr.Markdown()

        # Conectar eventos
        btn_analizar.click(
            fn=identificar_victima_y_agresor,
            inputs=[
                video_input, youtube_url,
                aggressor_threshold, victim_threshold,
                proximity_threshold, min_duration_frames,
                dist_threshold_px, speed_threshold,
                fist_distance_threshold, face_cover_threshold,
                crouch_threshold, movement_history,
                mostrar_interaccion
            ],
            outputs=[output_video, output_screenshot, output_text]
        )

        gr.Markdown("""
        ---
        ### ℹ️ Instrucciones:
        1. Sube un video o ingresa una URL de YouTube
        2. Ajusta los parámetros según necesites
        3. Haz clic en 'Iniciar Análisis'
        4. Revisa los resultados en las pestañas

        **Nota:** El procesamiento puede tomar varios minutos dependiendo de la duración del video.
        """)

    return demo

# ==========================================================================================================
# LANZAR APLICACIÓN
# ==========================================================================================================
# ==========================================================================================================
# LANZAR EN COLAB - VERSIÓN OPTIMIZADA
# ==========================================================================================================

if __name__ == "__main__":
    print("Iniciando aplicación...")

    # Crear interfaz
    demo = create_interface()

    # Detectar si estamos en Colab
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("🏁 Entorno: Google Colab detectado")
        # En Colab, usar share=True para obtener URL pública
        demo.launch(share=True, debug=False)
    else:
        print("🏁 Entorno: Local/Jupyter detectado")
        # En local, buscar puerto disponible
        import socket
        from contextlib import closing

        def find_free_port():
            with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
                s.bind(('', 0))
                s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                return s.getsockname()[1]

        free_port = find_free_port()
        print(f"✅ Usando puerto: {free_port}")

        demo.launch(
            server_name="0.0.0.0",
            server_port=free_port,
            share=False,
            inbrowser=True,
            show_error=True
        )


🚀 CARGANDO MODELOS
Cargando modelo YOLO...
✓ Modelo YOLO cargado
✓ Clasificador de violencia cargado

Iniciando aplicación...
🏁 Entorno: Google Colab detectado
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d72d99bd870cc64520.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [29]:
# ==========================================================================================================
# PARÁMETROS GLOBALES CONFIGURABLES (ajustados según recomendaciones)
# ==========================================================================================================
GLOBAL_PARAMS = {
    "AGGRESSOR_THRESHOLD": 2.8,
    "VICTIM_THRESHOLD": 1.5,
    "PROXIMITY_THRESHOLD": 140,
    "MIN_DURATION_FRAMES": 15,
    "dist_threshold_px": 85,
    "speed_threshold": 0.07,
    "FIST_DISTANCE_THRESHOLD": 0.12,
    "FACE_COVER_THRESHOLD": 0.10,
    "CROUCH_THRESHOLD": 0.18,
    "disappearance_threshold": 25,
    "movement_history": 15,
    "SMOOTH_WINDOW": 12,
}

def update_global_params(new_params):
    """Actualiza los parámetros globales"""
    GLOBAL_PARAMS.update(new_params)

# ==========================================================================================================
# UTILIDADES MEJORADAS
# ==========================================================================================================

def calculate_angle(a, b, c):
    try:
        a, b, c = np.array(a, dtype=float), np.array(b, dtype=float), np.array(c, dtype=float)
        ba = a - b
        bc = c - b
        denom = (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
        cosine_angle = np.dot(ba, bc) / denom
        cosine_angle = np.clip(cosine_angle, -1, 1)
        return np.degrees(np.arccos(cosine_angle))
    except Exception:
        return 180.0

def safe_kp(kp_list, idx):
    """Devuelve (x,y,conf) o [0,0,0]"""
    try:
        return kp_list[idx]
    except Exception:
        return np.array([0.0, 0.0, 0.0])

def detect_punch_direction(wrist, prev_wrist, target_torso, frame_scale=1.0):
    """Retorna intensidad estimada de golpe dirigido hacia target_torso"""
    try:
        if wrist[2] < 0.3 or prev_wrist[2] < 0.3:
            return 0.0
        v = np.array(wrist[:2]) - np.array(prev_wrist[:2])
        speed = np.linalg.norm(v)
        to_target = np.array(target_torso) - np.array(wrist[:2])
        if np.linalg.norm(to_target) == 0:
            return 0.0
        angle = calculate_angle([0, 0], v, to_target)
        if speed > 6 and angle < 35:
            return (speed / (frame_scale + 1e-6)) * 0.12
    except Exception:
        pass
    return 0.0

def detect_hook_punch(elbow, prev_elbow):
    """Detecta movimiento circular en codo (gancho)"""
    try:
        if elbow[2] < 0.3 or prev_elbow[2] < 0.3:
            return 0.0
        v = np.array(elbow[:2]) - np.array(prev_elbow[:2])
        angular_speed = np.linalg.norm(v)
        return 1.2 if angular_speed > 4 else 0.0
    except Exception:
        return 0.0

def detect_push_posture(keypoints, prev_keypoints, img_height):
    NOSE, L_HIP, R_HIP = 0, 11, 12
    try:
        if keypoints[NOSE][2] < 0.3 or keypoints[L_HIP][2] < 0.3 or keypoints[R_HIP][2] < 0.3:
            return 0.0
        torso_now = (keypoints[L_HIP][:2] + keypoints[R_HIP][:2]) / 2
        nose = np.array(keypoints[NOSE][:2])
        lean_dist = np.linalg.norm(nose - torso_now)
        if prev_keypoints is None or prev_keypoints[NOSE][2] < 0.3:
            return 0.0
        prev_nose = prev_keypoints[NOSE][:2]
        lean_speed = np.linalg.norm(nose - prev_nose)
        if lean_dist > 0.06 * img_height and lean_speed > 2:
            return lean_speed * 0.3
    except Exception:
        pass
    return 0.0

def compute_acceleration(history_deque):
    """Calcula aceleración desde historial de posiciones"""
    try:
        if len(history_deque) < 3:
            return 0.0
        p1 = np.array(history_deque[-3])
        p2 = np.array(history_deque[-2])
        p3 = np.array(history_deque[-1])
        v1 = np.linalg.norm(p2 - p1)
        v2 = np.linalg.norm(p3 - p2)
        return v2 - v1
    except Exception:
        return 0.0

def hand_to_body_distance(hand, target_box):
    try:
        hx, hy = hand[:2]
        tx1, ty1, w, h = target_box
        cx, cy = tx1 + w / 2.0, ty1 + h / 2.0
        return np.linalg.norm([hx - cx, hy - cy])
    except Exception:
        return float('inf')

def detect_collapse(keypoints, prev_keypoints):
    """Detecta caída brusca (knockdown)"""
    try:
        L_HIP, R_HIP = 11, 12
        if keypoints[L_HIP][2] < 0.3 or prev_keypoints is None or prev_keypoints[L_HIP][2] < 0.3:
            return 0.0
        hip_now = (keypoints[L_HIP][:2] + keypoints[R_HIP][:2]) / 2
        hip_prev = (prev_keypoints[L_HIP][:2] + prev_keypoints[R_HIP][:2]) / 2
        dy = hip_prev[1] - hip_now[1]
        if dy < -20:
            return abs(dy) * 0.05
    except Exception:
        pass
    return 0.0

# ==========================================================================================================
# DETECTORES DE POSTURAS MEJORADOS
# ==========================================================================================================

def detect_offensive_postures(keypoints, box, img_height, prev_keypoints=None, target_boxes=None):
    """Detección mejorada de posturas ofensivas"""
    features = {'arms_raised': 0.0, 'fists_closed': 0.0, 'forward_lean': 0.0,
                'offensive_gesture': 0.0, 'punch_direction': 0.0, 'hook': 0.0, 'push': 0.0,
                'total_score': 0.0}

    CONFIDENCE_THRESHOLD = 0.3
    FIST_DISTANCE_THRESHOLD = GLOBAL_PARAMS["FIST_DISTANCE_THRESHOLD"]
    NOSE, L_SHOULDER, R_SHOULDER = 0, 5, 6
    L_ELBOW, R_ELBOW, L_WRIST, R_WRIST = 7, 8, 9, 10

    try:
        # Brazos levantados
        if keypoints[L_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[L_SHOULDER][2] > CONFIDENCE_THRESHOLD:
            if keypoints[L_WRIST][1] < keypoints[L_SHOULDER][1]:
                features['arms_raised'] += 1.0
        if keypoints[R_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[R_SHOULDER][2] > CONFIDENCE_THRESHOLD:
            if keypoints[R_WRIST][1] < keypoints[R_SHOULDER][1]:
                features['arms_raised'] += 1.0

        # Puños cerrados
        if keypoints[L_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[L_ELBOW][2] > CONFIDENCE_THRESHOLD:
            dist_le = np.linalg.norm(keypoints[L_WRIST][:2] - keypoints[L_ELBOW][:2])
            if dist_le < FIST_DISTANCE_THRESHOLD * img_height:
                features['fists_closed'] += 1.5
        if keypoints[R_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[R_ELBOW][2] > CONFIDENCE_THRESHOLD:
            dist_re = np.linalg.norm(keypoints[R_WRIST][:2] - keypoints[R_ELBOW][:2])
            if dist_re < FIST_DISTANCE_THRESHOLD * img_height:
                features['fists_closed'] += 1.5

        # Inclinación hacia adelante
        if all(keypoints[i][2] > CONFIDENCE_THRESHOLD for i in [NOSE, L_SHOULDER, R_SHOULDER]):
            shoulder_center = (keypoints[L_SHOULDER][:2] + keypoints[R_SHOULDER][:2]) / 2.0
            if keypoints[NOSE][0] > shoulder_center[0]:
                features['forward_lean'] += 1.0

        # Gestos ofensivos por ángulos
        for shoulder, elbow, wrist in [(L_SHOULDER, L_ELBOW, L_WRIST), (R_SHOULDER, R_ELBOW, R_WRIST)]:
            if all(keypoints[i][2] > CONFIDENCE_THRESHOLD for i in [shoulder, elbow, wrist]):
                angle = calculate_angle(keypoints[shoulder][:2], keypoints[elbow][:2], keypoints[wrist][:2])
                if 150 < angle < 180:
                    features['offensive_gesture'] += 2.0

        # Detección de golpes dirigidos
        if prev_keypoints is not None and target_boxes is not None:
            for wrist_idx in [L_WRIST, R_WRIST]:
                prev_w = prev_keypoints[wrist_idx]
                curr_w = keypoints[wrist_idx]
                if curr_w[2] > CONFIDENCE_THRESHOLD and prev_w[2] > CONFIDENCE_THRESHOLD:
                    for tb in target_boxes:
                        center_tb = (tb[0] + tb[2] / 2.0, tb[1] + tb[3] / 2.0)
                        punch_score = detect_punch_direction(curr_w, prev_w, center_tb, frame_scale=img_height)
                        if punch_score > 0:
                            features['punch_direction'] += punch_score

                    # Detección de gancho
                    elbow_idx = L_ELBOW if wrist_idx == L_WRIST else R_ELBOW
                    prev_el = prev_keypoints[elbow_idx]
                    curr_el = keypoints[elbow_idx]
                    features['hook'] += detect_hook_punch(curr_el, prev_el)

        # Postura de empuje
        if prev_keypoints is not None:
            features['push'] += detect_push_posture(keypoints, prev_keypoints, img_height)

        features['total_score'] = (features['arms_raised'] + features['fists_closed'] +
                                   features['forward_lean'] + features['offensive_gesture'] +
                                   features['punch_direction'] + features['hook'] + features['push'])
    except Exception:
        pass

    return features

def detect_defensive_postures(keypoints, box, img_height, prev_keypoints=None):
    """Detección mejorada de posturas defensivas"""
    features = {'covering_face': 0.0, 'crouching': 0.0, 'defensive_arms': 0.0, 'collapse': 0.0, 'total_score': 0.0}

    CONFIDENCE_THRESHOLD = 0.3
    FACE_COVER_THRESHOLD = GLOBAL_PARAMS["FACE_COVER_THRESHOLD"]
    CROUCH_THRESHOLD = GLOBAL_PARAMS["CROUCH_THRESHOLD"]
    NOSE, L_EYE, R_EYE, L_EAR, R_EAR = 0, 1, 2, 3, 4
    L_SHOULDER, R_SHOULDER, L_ELBOW, R_ELBOW = 5, 6, 7, 8
    L_WRIST, R_WRIST, L_HIP, R_HIP = 9, 10, 11, 12

    try:
        # Protección facial
        face_points = [NOSE, L_EYE, R_EYE, L_EAR, R_EAR]
        valid_face_points = [keypoints[i][:2] for i in face_points if keypoints[i][2] > CONFIDENCE_THRESHOLD]
        if valid_face_points:
            face_center = np.mean(valid_face_points, axis=0)
            for wrist in [L_WRIST, R_WRIST]:
                if keypoints[wrist][2] > CONFIDENCE_THRESHOLD:
                    dist = np.linalg.norm(keypoints[wrist][:2] - face_center)
                    if dist < FACE_COVER_THRESHOLD * img_height:
                        features['covering_face'] += 2.0

        # Postura agachada
        if keypoints[L_SHOULDER][2] > CONFIDENCE_THRESHOLD and keypoints[L_HIP][2] > CONFIDENCE_THRESHOLD:
            upper_body_height = abs(keypoints[L_SHOULDER][1] - keypoints[L_HIP][1])
            if upper_body_height < CROUCH_THRESHOLD * img_height:
                features['crouching'] += 1.5

        # Brazos defensivos
        for shoulder, elbow, wrist in [(L_SHOULDER, L_ELBOW, L_WRIST), (R_SHOULDER, R_ELBOW, R_WRIST)]:
            if all(keypoints[i][2] > CONFIDENCE_THRESHOLD for i in [shoulder, elbow, wrist]):
                angle = calculate_angle(keypoints[shoulder][:2], keypoints[elbow][:2], keypoints[wrist][:2])
                if 60 < angle < 120:
                    features['defensive_arms'] += 1.0

        # Colapso/caída
        if prev_keypoints is not None:
            features['collapse'] += detect_collapse(keypoints, prev_keypoints)

        features['total_score'] = features['covering_face'] + features['crouching'] + features['defensive_arms'] + features['collapse']
    except Exception:
        pass

    return features

# ==========================================================================================================
# SISTEMA DE ANÁLISIS MEJORADO - VERSIÓN SIMPLIFICADA PERO FUNCIONAL
# ==========================================================================================================

def analyze_interactions_simple(person_boxes, keypoints_list, frame_height):
    """Versión simplificada pero funcional del análisis de interacciones"""
    interactions = {'proximity_scores': {}, 'aggression_vectors': {}}

    try:
        for i, (box_i, kp_i) in enumerate(zip(person_boxes, keypoints_list)):
            interactions['proximity_scores'][i] = 0.0
            interactions['aggression_vectors'][i] = 0.0

            center_i = np.array([box_i[0] + box_i[2]/2.0, box_i[1] + box_i[3]/2.0])
            offensive_score = detect_offensive_postures(kp_i, box_i, frame_height).get('total_score', 0.0)

            for j, (box_j, kp_j) in enumerate(zip(person_boxes, keypoints_list)):
                if i == j:
                    continue

                center_j = np.array([box_j[0] + box_j[2]/2.0, box_j[1] + box_j[3]/2.0])
                distance = np.linalg.norm(center_i - center_j)

                # Proximidad
                if distance < GLOBAL_PARAMS["PROXIMITY_THRESHOLD"]:
                    proximity_score = (GLOBAL_PARAMS["PROXIMITY_THRESHOLD"] - distance) / GLOBAL_PARAMS["PROXIMITY_THRESHOLD"]
                    interactions['proximity_scores'][i] += proximity_score

                # Agresión basada en postura ofensiva y proximidad
                if offensive_score > 1.0 and distance < 200:
                    interactions['aggression_vectors'][i] += offensive_score * 0.5

    except Exception as e:
        print(f"Error en analyze_interactions_simple: {e}")

    return interactions

def assign_roles_simple(person_boxes, keypoints_list, frame_height):
    """Asignación simplificada de roles que SÍ funciona"""
    roles = []

    try:
        if len(person_boxes) < 2:
            return roles

        interactions = analyze_interactions_simple(person_boxes, keypoints_list, frame_height)

        for i, (box, kp) in enumerate(zip(person_boxes, keypoints_list)):
            offensive_score = detect_offensive_postures(kp, box, frame_height).get('total_score', 0.0)
            defensive_score = detect_defensive_postures(kp, box, frame_height).get('total_score', 0.0)
            proximity_score = interactions['proximity_scores'].get(i, 0.0)
            aggression_score = interactions['aggression_vectors'].get(i, 0.0)

            # Puntuación final más simple pero efectiva
            final_score = (offensive_score * 1.5 + aggression_score * 1.2 +
                          proximity_score * 0.8 - defensive_score * 0.5)

            victim_score = (defensive_score * 1.5 + proximity_score * 0.6 -
                           offensive_score * 0.3)

            if final_score >= GLOBAL_PARAMS["AGGRESSOR_THRESHOLD"]:
                roles.append((i, 'agresor', final_score))
            elif victim_score >= GLOBAL_PARAMS["VICTIM_THRESHOLD"]:
                roles.append((i, 'victima', victim_score))
            else:
                roles.append((i, 'desconocido', final_score))

    except Exception as e:
        print(f"Error en assign_roles_simple: {e}")

    return roles

# ==========================================================================================================
# PROCESAMIENTO DE VIDEO MEJORADO - VERSIÓN QUE SÍ FUNCIONA
# ==========================================================================================================

def process_video_full_analysis(video_input, output_dir='outputs_rwf', max_frames=None, use_advanced=True, yolo_model=None, progress=None):
    """Función principal MEJORADA que SÍ detecta agresores y víctimas"""
    try:
        if progress is not None:
            progress(0, "🔄 Preparando video...")

        # Manejar entrada de video
        if isinstance(video_input, str) and os.path.exists(video_input):
            video_path = video_input
            video_name = os.path.basename(video_input).split('.')[0]
        else:
            with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as temp_file:
                if hasattr(video_input, 'read'):
                    temp_file.write(video_input.read())
                video_path = temp_file.name
            video_name = "video_temp"

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return 0, 0, 0, None, 0, 0, [], None, None, None

        fps = int(cap.get(cv2.CAP_PROP_FPS)) if cap.get(cv2.CAP_PROP_FPS) > 0 else 25
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # ✅ CORRECCIÓN IMPORTANTE: Procesar MÁS frames (no solo 50)
        if max_frames is None:
            max_frames = total_frames  # Procesar todo el video
        else:
            max_frames = min(max_frames, total_frames)  # Límite razonable

        # Configurar salida
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"{video_name}_processed.avi")
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (640, 480))

        # Variables de seguimiento MEJORADAS
        frame_count = 0
        processed_frame_idx = 0
        unique_aggressors = set()
        unique_victims = set()
        aggressor_frame_count = 0
        victim_frame_count = 0

        # Capturas
        first_agg_path = first_vic_path = first_both_path = None
        screenshot_saved_agg = screenshot_saved_vic = screenshot_saved_both = False

        if progress is not None:
            progress(0.1, "🎯 Iniciando análisis de frames...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count >= max_frames:
                break

            if progress is not None and frame_count % 10 == 0:
                progress(frame_count / max_frames, f"📊 Procesando frame {frame_count}/{max_frames}")

            # Redimensionar frame
            frame_resized = cv2.resize(frame, (640, 480))

            # Detectar personas con YOLO
            results = yolo_model(frame_resized, verbose=False)

            person_boxes = []
            keypoints_list = []
            current_aggressors = []
            current_victims = []

            if results and len(results) > 0:
                result = results[0]
                if result.boxes is not None and result.keypoints is not None:
                    for i, box in enumerate(result.boxes):
                        if int(box.cls) == 0:  # Solo personas
                            x1, y1, x2, y2 = box.xyxy[0].tolist()
                            conf = box.conf.item()
                            kp = result.keypoints.data[i].cpu().numpy() if i < len(result.keypoints.data) else np.zeros((17, 3))

                            person_boxes.append((x1, y1, x2 - x1, y2 - y1))
                            keypoints_list.append(kp)

            # ✅ ANÁLISIS MEJORADO: Usar el sistema de detección MEJORADO
            if len(person_boxes) >= 2:  # Solo analizar si hay al menos 2 personas
                roles = assign_roles_simple(person_boxes, keypoints_list, 480)

                for person_idx, role, score in roles:
                    if role == 'agresor':
                        current_aggressors.append(person_idx)
                        unique_aggressors.add(person_idx)
                        color = (0, 0, 255)  # Rojo
                        label = f"Agresor ({score:.1f})"
                    elif role == 'victima':
                        current_victims.append(person_idx)
                        unique_victims.add(person_idx)
                        color = (255, 0, 0)  # Azul
                        label = f"Victima ({score:.1f})"
                    else:
                        color = (0, 255, 0)  # Verde
                        label = "Desconocido"

                    # Dibujar bounding box y etiqueta
                    if person_idx < len(person_boxes):
                        x, y, w, h = [int(v) for v in person_boxes[person_idx]]
                        cv2.rectangle(frame_resized, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame_resized, label, (x, y - 10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Actualizar contadores
            if current_aggressors:
                aggressor_frame_count += 1
            if current_victims:
                victim_frame_count += 1

            # Guardar capturas MEJORADO
            if not screenshot_saved_agg and current_aggressors:
                first_agg_path = os.path.join(output_dir, f"{video_name}_primer_agresor.jpg")
                cv2.imwrite(first_agg_path, frame_resized)
                screenshot_saved_agg = True
                print(f"✅ Captura de agresor guardada")

            if not screenshot_saved_vic and current_victims:
                first_vic_path = os.path.join(output_dir, f"{video_name}_primera_victima.jpg")
                cv2.imwrite(first_vic_path, frame_resized)
                screenshot_saved_vic = True
                print(f"✅ Captura de víctima guardada")

            if not screenshot_saved_both and current_aggressors and current_victims:
                first_both_path = os.path.join(output_dir, f"{video_name}_primer_ambos.jpg")
                cv2.imwrite(first_both_path, frame_resized)
                screenshot_saved_both = True
                print(f"✅ Captura de interacción guardada")

            # Información en pantalla
            info_text = f"Frame: {frame_count} | Personas: {len(person_boxes)} | Agresores: {len(current_aggressors)} | Victimas: {len(current_victims)}"
            cv2.putText(frame_resized, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Escribir frame procesado
            out.write(frame_resized)
            frame_count += 1
            processed_frame_idx += 1

        # Liberar recursos
        cap.release()
        out.release()

        # Limpiar archivo temporal si se creó
        if 'temp_file' in locals():
            try:
                os.unlink(video_path)
            except:
                pass

        print(f"✅ Análisis completado: {len(unique_aggressors)} agresores, {len(unique_victims)} víctimas")

        if progress is not None:
            progress(1.0, "✅ Análisis completado!")

        return (
            len(unique_aggressors),
            len(unique_victims),
            processed_frame_idx,
            output_path,
            aggressor_frame_count,
            victim_frame_count,
            [],  # eventos
            first_agg_path,
            first_vic_path,
            first_both_path
        )

    except Exception as e:
        print(f"❌ Error en process_video_full_analysis: {e}")
        import traceback
        traceback.print_exc()
        return 0, 0, 0, None, 0, 0, [], None, None, None

# ==========================================================================================================
# EXTRACTOR DE FEATURES CORREGIDO
# ==========================================================================================================

def extract_lightweight_features(video_input, max_samples=10, yolo_model=None, progress=None):
    """Extrae features básicas para clasificación ML - VERSIÓN CORREGIDA"""
    try:
        if isinstance(video_input, str) and os.path.exists(video_input):
            video_path = video_input
        else:
            return np.zeros(6)  # Fallback

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return np.zeros(6)

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_frames = min(max_samples, total_frames)

        # Features básicas pero reales
        person_counts = []
        motion_values = []

        for i in range(sample_frames):
            ret, frame = cap.read()
            if not ret:
                break

            frame_resized = cv2.resize(frame, (416, 416))

            # Contar personas
            results = yolo_model(frame_resized, verbose=False, classes=[0])
            person_count = len(results[0].boxes) if results else 0
            person_counts.append(person_count)

            # Movimiento simple (diferencia de frames)
            if i > 0:
                gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                diff = cv2.absdiff(gray, prev_gray)
                motion_values.append(np.mean(diff))

            prev_frame = frame_resized.copy()

        cap.release()

        # Crear features
        features = np.array([
            np.mean(person_counts) if person_counts else 0,
            np.max(person_counts) if person_counts else 0,
            np.std(person_counts) if person_counts else 0,
            np.mean(motion_values) if motion_values else 0,
            np.max(motion_values) if motion_values else 0,
            total_frames / 1000.0  # Duración aproximada
        ])

        return features

    except Exception as e:
        print(f"Error en extract_lightweight_features: {e}")
        return np.zeros(6)

# ==========================================================================================================
# FUNCIÓN PRINCIPAL MEJORADA
# ==========================================================================================================

def identificar_victima_y_agresor(
    video_input=None,
    youtube_url="",
    aggressor_threshold=2.8,
    victim_threshold=1.5,
    proximity_threshold=140,
    min_duration_frames=15,
    dist_threshold_px=85,
    speed_threshold=0.04,
    fist_distance_threshold=0.08,
    face_cover_threshold=0.10,
    crouch_threshold=0.25,
    movement_history=15,
    mostrar_interaccion="Ambos",
    progress=gr.Progress()
):
    """Función principal MEJORADA con detección REAL"""

    try:
        # Actualizar parámetros globales con tus valores MEJORADOS
        update_global_params({
            "AGGRESSOR_THRESHOLD": aggressor_threshold,
            "VICTIM_THRESHOLD": victim_threshold,
            "PROXIMITY_THRESHOLD": proximity_threshold,
            "MIN_DURATION_FRAMES": min_duration_frames,
            "dist_threshold_px": dist_threshold_px,
            "speed_threshold": speed_threshold,
            "FIST_DISTANCE_THRESHOLD": fist_distance_threshold,
            "FACE_COVER_THRESHOLD": face_cover_threshold,
            "CROUCH_THRESHOLD": crouch_threshold,
            "movement_history": movement_history,
        })

        video_path = None
        temp_file = None

        # Determinar fuente de video
        if youtube_url and youtube_url.strip():
            progress(0.1, "🌐 Descargando video de YouTube...")
            video_path = descargar_youtube(youtube_url, progress)
            temp_file = video_path
        elif video_input is not None:
            video_path = video_input
        else:
            raise gr.Error("❌ Por favor, sube un video o ingresa una URL de YouTube")

        progress(0.3, "🎯 Iniciando análisis avanzado del video...")

        # ✅ CORRECCIÓN: Procesar MÁS frames (no solo 50)
        (
            num_agresores,
            num_victimas,
            frames_procesados,
            video_procesado_path,
            frames_con_agresor,
            frames_con_victima,
            eventos,
            img_agresor,
            img_victima,
            img_ambos
        ) = process_video_full_analysis(
            video_path,
            output_dir='outputs_rwf',
            max_frames=None,  # ✅ Procesar TODO el video
            use_advanced=True,
            yolo_model=yolo_model,
            progress=progress
        )

        progress(0.9, "📊 Generando resultados...")

        # Seleccionar imagen para mostrar
        img_mostrar = None
        mensaje_captura = ""

        if mostrar_interaccion == "Agresor" and img_agresor and os.path.exists(img_agresor):
            img_mostrar = img_agresor
            mensaje_captura = "🔴 Primera detección de AGRESOR"
        elif mostrar_interaccion == "Víctima" and img_victima and os.path.exists(img_victima):
            img_mostrar = img_victima
            mensaje_captura = "🔵 Primera detección de VÍCTIMA"
        elif mostrar_interaccion == "Ambos" and img_ambos and os.path.exists(img_ambos):
            img_mostrar = img_ambos
            mensaje_captura = "👥 Primera interacción AGRESOR + VÍCTIMA"
        else:
            mensaje_captura = "⚠ No se capturaron interacciones del tipo seleccionado"

        # ✅ REPORTE MEJORADO con información REAL
        violencia_detectada = (frames_con_agresor >= min_duration_frames and
                              frames_con_victima >= min_duration_frames)

        reporte = f"""
## 📊 Resultados del Análisis

### 👥 Detección de Personas
- **Agresores detectados:** {num_agresores}
- **Víctimas detectadas:** {num_victimas}
- **Frames procesados:** {frames_procesados}

### 🎯 Actividad Detectada
- **Frames con agresor:** {frames_con_agresor}
- **Frames con víctima:** {frames_con_victima}
- **Violencia prolongada:** {'✅ SÍ' if violencia_detectada else '❌ NO'}

### 📸 Capturas
- **{mensaje_captura}**

### ⚙️ Configuración Usada
- **Umbral agresor:** {aggressor_threshold}
- **Umbral víctima:** {victim_threshold}
- **Proximidad:** {proximity_threshold}px
- **Duración mínima:** {min_duration_frames} frames
"""

        # Clasificación ML MEJORADA (manejo de errores)
        if violence_classifier is not None:
            try:
                features = extract_lightweight_features(video_path, yolo_model=yolo_model)
                # ✅ CORRECCIÓN: Manejar el error de índice
                prediccion = violence_classifier.predict([features])[0]
                probabilidad = violence_classifier.predict_proba([features])[0]

                # Verificar la forma de las probabilidades
                if len(probabilidad) >= 2:
                    if prediccion == 1:
                        reporte += f"\n### 🔴 CLASIFICADOR ML: VIOLENCIA DETECTADA\n- **Confianza:** {probabilidad[1]:.1%}"
                    else:
                        reporte += f"\n### ✅ CLASIFICADOR ML: NO VIOLENCIA\n- **Confianza:** {probabilidad[0]:.1%}"
                else:
                    reporte += f"\n### ⚠ CLASIFICADOR ML: Resultado - {prediccion}"

            except Exception as e:
                reporte += f"\n### ⚠ Clasificación ML no disponible\n- Error: {str(e)}"
        else:
            reporte += "\n### ℹ️ Clasificador ML no cargado"

        # Limpiar archivos temporales
        if temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except:
                pass

        progress(1.0, "✅ ¡Análisis completado!")

        return video_procesado_path, img_mostrar, reporte

    except Exception as e:
        import traceback
        error_msg = f"Error durante el procesamiento: {str(e)}"
        print(f"ERROR: {error_msg}")
        print(traceback.format_exc())
        raise gr.Error(error_msg)

# === LANZAR ===
print("Iniciando Identificador Víctima y Agresor...")

try:
    demo.launch(
        share=True,                    # mantiene el link público temporal (opcional)
        debug=False,                   # <- evita que la celda quede "colgada" en debug mode
        prevent_thread_lock=True,      # <- crucial para Colab / notebooks
        server_name="0.0.0.0",
        server_port=7860,
        inbrowser=False,               # en Colab conviene False; cambia a True si ejecutas localmente
        allowed_paths=["/content", "./"]
    )
except KeyboardInterrupt:
    # se alcanza si el usuario detiene la celda; cerramos de forma limpia
    print("Servidor detenido por el usuario (KeyboardInterrupt).")
except Exception as e:
    # atrapa otros errores, útil para depuración en notebooks
    print(f"Error al lanzar Gradio: {e}")


# ==========================================================================================================
# CARGAR MODELOS
# ==========================================================================================================

print("\n" + "="*80)
print("🚀 CARGANDO MODELOS")
print("="*80)

print("Cargando modelo YOLO...")
try:
    yolo_model = YOLO('yolov8n-pose.pt')
    print("✓ Modelo YOLO cargado")
except Exception as e:
    print(f"❌ Error cargando YOLO: {e}")
    yolo_model = None

# Intentar cargar clasificador de violencia
model_path = 'violence_classifier.pkl'
violence_classifier = None
if os.path.exists(model_path):
    try:
        violence_classifier = joblib.load(model_path)
        print("✓ Clasificador de violencia cargado")
    except Exception as e:
        print(f"⚠ Error cargando clasificador: {e}")
else:
    print("⚠ No se encontró violence_classifier.pkl - El sistema funcionará sin clasificación ML")

print("="*80 + "\n")

# ==========================================================================================================
# INTERFAZ GRADIO
# ==========================================================================================================

def create_interface():
    """Crea y retorna la interfaz de Gradio"""

    with gr.Blocks(theme=gr.themes.Soft(), title="Identificador Víctima y Agresor") as demo:
        gr.Markdown("""
        # 🎯 Identificador Víctima y Agresor
        ### Sistema de Detección de Violencia con IA
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📁 Fuente de Video")

                with gr.Tab("Subir Video"):
                    video_input = gr.Video(label="Seleccionar video", sources=["upload"])

                with gr.Tab("YouTube URL"):
                    youtube_url = gr.Textbox(
                        label="URL de YouTube",
                        placeholder="https://www.youtube.com/watch?v=...",
                        lines=1
                    )

                gr.Markdown("### ⚙️ Parámetros de Detección")

                with gr.Accordion("Parámetros Principales", open=True):
                    aggressor_threshold = gr.Slider(
                        minimum=0.5, maximum=5.0, value=2.8, step=0.1,
                        label="Umbral Agresor"
                    )
                    victim_threshold = gr.Slider(
                        minimum=0.5, maximum=5.0, value=1.5, step=0.1,
                        label="Umbral Víctima"
                    )
                    proximity_threshold = gr.Slider(
                        minimum=50, maximum=300, value=140, step=10,
                        label="Umbral de Proximidad (px)"
                    )
                    min_duration_frames = gr.Slider(
                        minimum=5, maximum=100, value=15, step=5,
                        label="Duración Mínima (frames)"
                    )

                with gr.Accordion("Parámetros Avanzados", open=False):
                    dist_threshold_px = gr.Slider(
                        minimum=20, maximum=200, value=85, step=5,
                        label="Distancia para Golpes (px)"
                    )
                    speed_threshold = gr.Slider(
                        minimum=0.01, maximum=0.2, value=0.04, step=0.01,
                        label="Umbral de Velocidad"
                    )
                    fist_distance_threshold = gr.Slider(
                        minimum=0.05, maximum=0.3, value=0.08, step=0.01,
                        label="Detección de Puños"
                    )
                    face_cover_threshold = gr.Slider(
                        minimum=0.05, maximum=0.3, value=0.10, step=0.01,
                        label="Protección Facial"
                    )
                    crouch_threshold = gr.Slider(
                        minimum=0.1, maximum=0.5, value=0.25, step=0.01,
                        label="Postura Agachada"
                    )
                    movement_history = gr.Slider(
                        minimum=5, maximum=50, value=15, step=5,
                        label="Historial de Movimiento"
                    )

                mostrar_interaccion = gr.Radio(
                    choices=["Ambos", "Agresor", "Víctima"],
                    value="Ambos",
                    label="📸 Mostrar en Resultados"
                )

                btn_analizar = gr.Button(
                    "🚀 Iniciar Análisis",
                    variant="primary",
                    size="lg"
                )

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Resultados")

                with gr.Tab("Video Procesado"):
                    output_video = gr.Video(label="Video Analizado")

                with gr.Tab("Captura"):
                    output_screenshot = gr.Image(label="Primera Interacción Detectada")

                with gr.Tab("Reporte"):
                    output_text = gr.Markdown()

        # Conectar eventos
        btn_analizar.click(
            fn=identificar_victima_y_agresor,
            inputs=[
                video_input, youtube_url,
                aggressor_threshold, victim_threshold,
                proximity_threshold, min_duration_frames,
                dist_threshold_px, speed_threshold,
                fist_distance_threshold, face_cover_threshold,
                crouch_threshold, movement_history,
                mostrar_interaccion
            ],
            outputs=[output_video, output_screenshot, output_text]
        )

        gr.Markdown("""
        ---
        ### ℹ️ Instrucciones:
        1. Sube un video o ingresa una URL de YouTube
        2. Ajusta los parámetros según necesites
        3. Haz clic en 'Iniciar Análisis'
        4. Revisa los resultados en las pestañas

        **Nota:** El procesamiento puede tomar varios minutos dependiendo de la duración del video.
        """)

    return demo

# ==========================================================================================================
# LANZAR APLICACIÓN
# ==========================================================================================================
# ==========================================================================================================
# LANZAR EN COLAB - VERSIÓN OPTIMIZADA
# ==========================================================================================================

if __name__ == "__main__":
    print("Iniciando aplicación...")

    # Crear interfaz
    demo = create_interface()

    # Detectar si estamos en Colab
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        print("🏁 Entorno: Google Colab detectado")
        # En Colab, usar share=True para obtener URL pública
        demo.launch(share=True, debug=False)
    else:
        print("🏁 Entorno: Local/Jupyter detectado")
        # En local, buscar puerto disponible
        import socket
        from contextlib import closing

        def find_free_port():
            with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
                s.bind(('', 0))
                s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                return s.getsockname()[1]

        free_port = find_free_port()
        print(f"✅ Usando puerto: {free_port}")

        demo.launch(
            server_name="0.0.0.0",
            server_port=free_port,
            share=False,
            inbrowser=True,
            show_error=True
        )

Iniciando Identificador Víctima y Agresor...
Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5bdbdae6dfd730e39.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



🚀 CARGANDO MODELOS
Cargando modelo YOLO...
✓ Modelo YOLO cargado
✓ Clasificador de violencia cargado

Iniciando aplicación...
🏁 Entorno: Google Colab detectado
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://311e42a44f77a09641.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Modelo que procesa videos completos

In [38]:
"""violence_pipeline_mejorado.py - Sistema completo de detección de violencia
Incluye interfaz Gradio funcional con análisis ML y detección de poses
"""

import gradio as gr
import yt_dlp
import shutil
from pathlib import Path
import os
import tempfile
import json
import time
from collections import defaultdict, deque

import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

import logging

# Configurar logger
logging.basicConfig(
    filename='app.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ==========================================================================================================
# PARÁMETROS GLOBALES CONFIGURABLES (ajustados según recomendaciones)
# ==========================================================================================================
GLOBAL_PARAMS = {
    "AGGRESSOR_THRESHOLD": 2.8,
    "VICTIM_THRESHOLD": 1.5,
    "PROXIMITY_THRESHOLD": 140,
    "MIN_DURATION_FRAMES": 15,
    "dist_threshold_px": 85,
    "speed_threshold": 0.07,
    "FIST_DISTANCE_THRESHOLD": 0.12,
    "FACE_COVER_THRESHOLD": 0.10,
    "CROUCH_THRESHOLD": 0.18,
    "disappearance_threshold": 25,
    "movement_history": 15,
    "SMOOTH_WINDOW": 12,
}

def update_global_params(new_params):
    """Actualiza los parámetros globales"""
    GLOBAL_PARAMS.update(new_params)

# ==========================================================================================================
# UTILIDADES MEJORADAS
# ==========================================================================================================

def calculate_angle(a, b, c):
    try:
        a, b, c = np.array(a, dtype=float), np.array(b, dtype=float), np.array(c, dtype=float)
        ba = a - b
        bc = c - b
        denom = (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
        cosine_angle = np.dot(ba, bc) / denom
        cosine_angle = np.clip(cosine_angle, -1, 1)
        return np.degrees(np.arccos(cosine_angle))
    except Exception:
        return 180.0

def safe_kp(kp_list, idx):
    """Devuelve (x,y,conf) o [0,0,0]"""
    try:
        return kp_list[idx]
    except Exception:
        return np.array([0.0, 0.0, 0.0])

def detect_punch_direction(wrist, prev_wrist, target_torso, frame_scale=1.0):
    """Retorna intensidad estimada de golpe dirigido hacia target_torso"""
    try:
        if wrist[2] < 0.3 or prev_wrist[2] < 0.3:
            return 0.0
        v = np.array(wrist[:2]) - np.array(prev_wrist[:2])
        speed = np.linalg.norm(v)
        to_target = np.array(target_torso) - np.array(wrist[:2])
        if np.linalg.norm(to_target) == 0:
            return 0.0
        angle = calculate_angle([0, 0], v, to_target)
        if speed > 6 and angle < 35:
            return (speed / (frame_scale + 1e-6)) * 0.12
    except Exception:
        pass
    return 0.0

def detect_hook_punch(elbow, prev_elbow):
    """Detecta movimiento circular en codo (gancho)"""
    try:
        if elbow[2] < 0.3 or prev_elbow[2] < 0.3:
            return 0.0
        v = np.array(elbow[:2]) - np.array(prev_elbow[:2])
        angular_speed = np.linalg.norm(v)
        return 1.2 if angular_speed > 4 else 0.0
    except Exception:
        return 0.0

def detect_push_posture(keypoints, prev_keypoints, img_height):
    NOSE, L_HIP, R_HIP = 0, 11, 12
    try:
        if keypoints[NOSE][2] < 0.3 or keypoints[L_HIP][2] < 0.3 or keypoints[R_HIP][2] < 0.3:
            return 0.0
        torso_now = (keypoints[L_HIP][:2] + keypoints[R_HIP][:2]) / 2
        nose = np.array(keypoints[NOSE][:2])
        lean_dist = np.linalg.norm(nose - torso_now)
        if prev_keypoints is None or prev_keypoints[NOSE][2] < 0.3:
            return 0.0
        prev_nose = prev_keypoints[NOSE][:2]
        lean_speed = np.linalg.norm(nose - prev_nose)
        if lean_dist > 0.06 * img_height and lean_speed > 2:
            return lean_speed * 0.3
    except Exception:
        pass
    return 0.0

def compute_acceleration(history_deque):
    """Calcula aceleración desde historial de posiciones"""
    try:
        if len(history_deque) < 3:
            return 0.0
        p1 = np.array(history_deque[-3])
        p2 = np.array(history_deque[-2])
        p3 = np.array(history_deque[-1])
        v1 = np.linalg.norm(p2 - p1)
        v2 = np.linalg.norm(p3 - p2)
        return v2 - v1
    except Exception:
        return 0.0

def hand_to_body_distance(hand, target_box):
    try:
        hx, hy = hand[:2]
        tx1, ty1, w, h = target_box
        cx, cy = tx1 + w / 2.0, ty1 + h / 2.0
        return np.linalg.norm([hx - cx, hy - cy])
    except Exception:
        return float('inf')

def detect_collapse(keypoints, prev_keypoints):
    """Detecta caída brusca (knockdown)"""
    try:
        L_HIP, R_HIP = 11, 12
        if keypoints[L_HIP][2] < 0.3 or prev_keypoints is None or prev_keypoints[L_HIP][2] < 0.3:
            return 0.0
        hip_now = (keypoints[L_HIP][:2] + keypoints[R_HIP][:2]) / 2
        hip_prev = (prev_keypoints[L_HIP][:2] + prev_keypoints[R_HIP][:2]) / 2
        dy = hip_prev[1] - hip_now[1]
        if dy < -20:
            return abs(dy) * 0.05
    except Exception:
        pass
    return 0.0

# ==========================================================================================================
# DETECTORES DE POSTURAS MEJORADOS
# ==========================================================================================================

def detect_offensive_postures(keypoints, box, img_height, prev_keypoints=None, target_boxes=None):
    """Detección mejorada de posturas ofensivas"""
    features = {'arms_raised': 0.0, 'fists_closed': 0.0, 'forward_lean': 0.0,
                'offensive_gesture': 0.0, 'punch_direction': 0.0, 'hook': 0.0, 'push': 0.0,
                'total_score': 0.0}

    CONFIDENCE_THRESHOLD = 0.3
    FIST_DISTANCE_THRESHOLD = GLOBAL_PARAMS["FIST_DISTANCE_THRESHOLD"]
    NOSE, L_SHOULDER, R_SHOULDER = 0, 5, 6
    L_ELBOW, R_ELBOW, L_WRIST, R_WRIST = 7, 8, 9, 10

    try:
        # Brazos levantados
        if keypoints[L_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[L_SHOULDER][2] > CONFIDENCE_THRESHOLD:
            if keypoints[L_WRIST][1] < keypoints[L_SHOULDER][1]:
                features['arms_raised'] += 1.0
        if keypoints[R_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[R_SHOULDER][2] > CONFIDENCE_THRESHOLD:
            if keypoints[R_WRIST][1] < keypoints[R_SHOULDER][1]:
                features['arms_raised'] += 1.0

        # Puños cerrados
        if keypoints[L_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[L_ELBOW][2] > CONFIDENCE_THRESHOLD:
            dist_le = np.linalg.norm(keypoints[L_WRIST][:2] - keypoints[L_ELBOW][:2])
            if dist_le < FIST_DISTANCE_THRESHOLD * img_height:
                features['fists_closed'] += 1.5
        if keypoints[R_WRIST][2] > CONFIDENCE_THRESHOLD and keypoints[R_ELBOW][2] > CONFIDENCE_THRESHOLD:
            dist_re = np.linalg.norm(keypoints[R_WRIST][:2] - keypoints[R_ELBOW][:2])
            if dist_re < FIST_DISTANCE_THRESHOLD * img_height:
                features['fists_closed'] += 1.5

        # Inclinación hacia adelante
        if all(keypoints[i][2] > CONFIDENCE_THRESHOLD for i in [NOSE, L_SHOULDER, R_SHOULDER]):
            shoulder_center = (keypoints[L_SHOULDER][:2] + keypoints[R_SHOULDER][:2]) / 2.0
            if keypoints[NOSE][0] > shoulder_center[0]:
                features['forward_lean'] += 1.0

        # Gestos ofensivos por ángulos
        for shoulder, elbow, wrist in [(L_SHOULDER, L_ELBOW, L_WRIST), (R_SHOULDER, R_ELBOW, R_WRIST)]:
            if all(keypoints[i][2] > CONFIDENCE_THRESHOLD for i in [shoulder, elbow, wrist]):
                angle = calculate_angle(keypoints[shoulder][:2], keypoints[elbow][:2], keypoints[wrist][:2])
                if 150 < angle < 180:
                    features['offensive_gesture'] += 2.0

        # Detección de golpes dirigidos
        if prev_keypoints is not None and target_boxes is not None:
            for wrist_idx in [L_WRIST, R_WRIST]:
                prev_w = prev_keypoints[wrist_idx]
                curr_w = keypoints[wrist_idx]
                if curr_w[2] > CONFIDENCE_THRESHOLD and prev_w[2] > CONFIDENCE_THRESHOLD:
                    for tb in target_boxes:
                        center_tb = (tb[0] + tb[2] / 2.0, tb[1] + tb[3] / 2.0)
                        punch_score = detect_punch_direction(curr_w, prev_w, center_tb, frame_scale=img_height)
                        if punch_score > 0:
                            features['punch_direction'] += punch_score

                    # Detección de gancho
                    elbow_idx = L_ELBOW if wrist_idx == L_WRIST else R_ELBOW
                    prev_el = prev_keypoints[elbow_idx]
                    curr_el = keypoints[elbow_idx]
                    features['hook'] += detect_hook_punch(curr_el, prev_el)

        # Postura de empuje
        if prev_keypoints is not None:
            features['push'] += detect_push_posture(keypoints, prev_keypoints, img_height)

        features['total_score'] = (features['arms_raised'] + features['fists_closed'] +
                                   features['forward_lean'] + features['offensive_gesture'] +
                                   features['punch_direction'] + features['hook'] + features['push'])
    except Exception:
        pass

    return features

def detect_defensive_postures(keypoints, box, img_height, prev_keypoints=None):
    """Detección mejorada de posturas defensivas"""
    features = {'covering_face': 0.0, 'crouching': 0.0, 'defensive_arms': 0.0, 'collapse': 0.0, 'total_score': 0.0}

    CONFIDENCE_THRESHOLD = 0.3
    FACE_COVER_THRESHOLD = GLOBAL_PARAMS["FACE_COVER_THRESHOLD"]
    CROUCH_THRESHOLD = GLOBAL_PARAMS["CROUCH_THRESHOLD"]
    NOSE, L_EYE, R_EYE, L_EAR, R_EAR = 0, 1, 2, 3, 4
    L_SHOULDER, R_SHOULDER, L_ELBOW, R_ELBOW = 5, 6, 7, 8
    L_WRIST, R_WRIST, L_HIP, R_HIP = 9, 10, 11, 12

    try:
        # Protección facial
        face_points = [NOSE, L_EYE, R_EYE, L_EAR, R_EAR]
        valid_face_points = [keypoints[i][:2] for i in face_points if keypoints[i][2] > CONFIDENCE_THRESHOLD]
        if valid_face_points:
            face_center = np.mean(valid_face_points, axis=0)
            for wrist in [L_WRIST, R_WRIST]:
                if keypoints[wrist][2] > CONFIDENCE_THRESHOLD:
                    dist = np.linalg.norm(keypoints[wrist][:2] - face_center)
                    if dist < FACE_COVER_THRESHOLD * img_height:
                        features['covering_face'] += 2.0

        # Postura agachada
        if keypoints[L_SHOULDER][2] > CONFIDENCE_THRESHOLD and keypoints[L_HIP][2] > CONFIDENCE_THRESHOLD:
            upper_body_height = abs(keypoints[L_SHOULDER][1] - keypoints[L_HIP][1])
            if upper_body_height < CROUCH_THRESHOLD * img_height:
                features['crouching'] += 1.5

        # Brazos defensivos
        for shoulder, elbow, wrist in [(L_SHOULDER, L_ELBOW, L_WRIST), (R_SHOULDER, R_ELBOW, R_WRIST)]:
            if all(keypoints[i][2] > CONFIDENCE_THRESHOLD for i in [shoulder, elbow, wrist]):
                angle = calculate_angle(keypoints[shoulder][:2], keypoints[elbow][:2], keypoints[wrist][:2])
                if 60 < angle < 120:
                    features['defensive_arms'] += 1.0

        # Colapso/caída
        if prev_keypoints is not None:
            features['collapse'] += detect_collapse(keypoints, prev_keypoints)

        features['total_score'] = features['covering_face'] + features['crouching'] + features['defensive_arms'] + features['collapse']
    except Exception:
        pass

    return features

# ==========================================================================================================
# SISTEMA DE ANÁLISIS MEJORADO - VERSIÓN SIMPLIFICADA PERO FUNCIONAL
# ==========================================================================================================

def analyze_interactions_simple(person_boxes, keypoints_list, frame_height):
    """Versión simplificada pero funcional del análisis de interacciones"""
    interactions = {'proximity_scores': {}, 'aggression_vectors': {}}

    try:
        for i, (box_i, kp_i) in enumerate(zip(person_boxes, keypoints_list)):
            interactions['proximity_scores'][i] = 0.0
            interactions['aggression_vectors'][i] = 0.0

            center_i = np.array([box_i[0] + box_i[2]/2.0, box_i[1] + box_i[3]/2.0])
            offensive_score = detect_offensive_postures(kp_i, box_i, frame_height).get('total_score', 0.0)

            for j, (box_j, kp_j) in enumerate(zip(person_boxes, keypoints_list)):
                if i == j:
                    continue

                center_j = np.array([box_j[0] + box_j[2]/2.0, box_j[1] + box_j[3]/2.0])
                distance = np.linalg.norm(center_i - center_j)

                # Proximidad
                if distance < GLOBAL_PARAMS["PROXIMITY_THRESHOLD"]:
                    proximity_score = (GLOBAL_PARAMS["PROXIMITY_THRESHOLD"] - distance) / GLOBAL_PARAMS["PROXIMITY_THRESHOLD"]
                    interactions['proximity_scores'][i] += proximity_score

                # Agresión basada en postura ofensiva y proximidad
                if offensive_score > 1.0 and distance < 200:
                    interactions['aggression_vectors'][i] += offensive_score * 0.5

    except Exception as e:
        logger.error(f"Error en analyze_interactions_simple: {e}")

    return interactions

def assign_roles_simple(person_boxes, keypoints_list, frame_height):
    """Asignación simplificada de roles que SÍ funciona"""
    roles = []

    try:
        if len(person_boxes) < 2:
            return roles

        interactions = analyze_interactions_simple(person_boxes, keypoints_list, frame_height)

        for i, (box, kp) in enumerate(zip(person_boxes, keypoints_list)):
            offensive_score = detect_offensive_postures(kp, box, frame_height).get('total_score', 0.0)
            defensive_score = detect_defensive_postures(kp, box, frame_height).get('total_score', 0.0)
            proximity_score = interactions['proximity_scores'].get(i, 0.0)
            aggression_score = interactions['aggression_vectors'].get(i, 0.0)

            # Puntuación final más simple pero efectiva
            final_score = (offensive_score * 1.5 + aggression_score * 1.2 +
                          proximity_score * 0.8 - defensive_score * 0.5)

            victim_score = (defensive_score * 1.5 + proximity_score * 0.6 -
                           offensive_score * 0.3)

            if final_score >= GLOBAL_PARAMS["AGGRESSOR_THRESHOLD"]:
                roles.append((i, 'agresor', final_score))
            elif victim_score >= GLOBAL_PARAMS["VICTIM_THRESHOLD"]:
                roles.append((i, 'victima', victim_score))
            else:
                roles.append((i, 'desconocido', final_score))

    except Exception as e:
        logger.error(f"Error en assign_roles_simple: {e}")

    return roles

# ==========================================================================================================
# PROCESAMIENTO DE VIDEO MEJORADO - VERSIÓN QUE SÍ FUNCIONA
# ==========================================================================================================

def process_video_full_analysis(video_input, output_dir='outputs_rwf', max_frames=None, use_advanced=True, yolo_model=None, progress=None, frame_skip=2):
    """Función principal MEJORADA que SÍ detecta agresores y víctimas"""
    try:
        if progress is not None:
            progress(0, "🔄 Preparando video...")

        # Manejar entrada de video
        if isinstance(video_input, str) and os.path.exists(video_input):
            video_path = video_input
            video_name = os.path.basename(video_input).split('.')[0]
        else:
            with tempfile.NamedTemporaryFile(suffix='.mp4', delete=False) as temp_file:
                if hasattr(video_input, 'read'):
                    temp_file.write(video_input.read())
                video_path = temp_file.name
            video_name = "video_temp"

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return 0, 0, 0, None, 0, 0, [], None, None, None

        fps = int(cap.get(cv2.CAP_PROP_FPS)) if cap.get(cv2.CAP_PROP_FPS) > 0 else 25
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Procesar TODO el video
        max_frames = total_frames

        # Configurar salida con MP4 para compatibilidad
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"{video_name}_processed.mp4")
        out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (640, 480))

        # Variables de seguimiento MEJORADAS
        frame_count = 0
        processed_frame_idx = 0
        unique_aggressors = set()
        unique_victims = set()
        aggressor_frame_count = 0
        victim_frame_count = 0

        # Capturas
        first_agg_path = first_vic_path = first_both_path = None
        screenshot_saved_agg = screenshot_saved_vic = screenshot_saved_both = False

        if progress is not None:
            progress(0.1, "🎯 Iniciando análisis de frames...")

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret or frame_count >= max_frames:
                break

            if frame_count % frame_skip != 0:
                frame_count += 1
                continue

            if progress is not None and frame_count % 10 == 0:
                progress(frame_count / max_frames, f"📊 Procesando frame {frame_count}/{max_frames}")

            # Redimensionar frame
            frame_resized = cv2.resize(frame, (640, 480))

            # Detectar personas con YOLO
            results = yolo_model(frame_resized, verbose=False)

            person_boxes = []
            keypoints_list = []
            current_aggressors = []
            current_victims = []

            if results and len(results) > 0:
                result = results[0]
                if result.boxes is not None and result.keypoints is not None:
                    for i, box in enumerate(result.boxes):
                        if int(box.cls) == 0:  # Solo personas
                            x1, y1, x2, y2 = box.xyxy[0].tolist()
                            conf = box.conf.item()
                            kp = result.keypoints.data[i].cpu().numpy() if i < len(result.keypoints.data) else np.zeros((17, 3))

                            person_boxes.append((x1, y1, x2 - x1, y2 - y1))
                            keypoints_list.append(kp)

            # ✅ ANÁLISIS MEJORADO: Usar el sistema de detección MEJORADO
            if len(person_boxes) >= 2:  # Solo analizar si hay al menos 2 personas
                roles = assign_roles_simple(person_boxes, keypoints_list, 480)

                for person_idx, role, score in roles:
                    if role == 'agresor':
                        current_aggressors.append(person_idx)
                        unique_aggressors.add(person_idx)
                        color = (0, 0, 255)  # Rojo
                        label = f"Agresor ({score:.1f})"
                    elif role == 'victima':
                        current_victims.append(person_idx)
                        unique_victims.add(person_idx)
                        color = (255, 0, 0)  # Azul
                        label = f"Victima ({score:.1f})"
                    else:
                        color = (0, 255, 0)  # Verde
                        label = "Desconocido"

                    # Dibujar bounding box y etiqueta
                    if person_idx < len(person_boxes):
                        x, y, w, h = [int(v) for v in person_boxes[person_idx]]
                        cv2.rectangle(frame_resized, (x, y), (x + w, y + h), color, 2)
                        cv2.putText(frame_resized, label, (x, y - 10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

            # Actualizar contadores
            if current_aggressors:
                aggressor_frame_count += 1
            if current_victims:
                victim_frame_count += 1

            # Guardar capturas MEJORADO
            if not screenshot_saved_agg and current_aggressors:
                first_agg_path = os.path.join(output_dir, f"{video_name}_primer_agresor.jpg")
                cv2.imwrite(first_agg_path, frame_resized)
                screenshot_saved_agg = True
                logger.info(f"✅ Captura de agresor guardada")

            if not screenshot_saved_vic and current_victims:
                first_vic_path = os.path.join(output_dir, f"{video_name}_primera_victima.jpg")
                cv2.imwrite(first_vic_path, frame_resized)
                screenshot_saved_vic = True
                logger.info(f"✅ Captura de víctima guardada")

            if not screenshot_saved_both and current_aggressors and current_victims:
                first_both_path = os.path.join(output_dir, f"{video_name}_primer_ambos.jpg")
                cv2.imwrite(first_both_path, frame_resized)
                screenshot_saved_both = True
                logger.info(f"✅ Captura de interacción guardada")

            # Información en pantalla
            info_text = f"Frame: {frame_count} | Personas: {len(person_boxes)} | Agresores: {len(current_aggressors)} | Victimas: {len(current_victims)}"
            cv2.putText(frame_resized, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Escribir frame procesado
            out.write(frame_resized)
            frame_count += 1
            processed_frame_idx += 1

        # Liberar recursos
        cap.release()
        out.release()

        # Limpiar archivo temporal si se creó
        if 'temp_file' in locals():
            try:
                os.unlink(video_path)
            except:
                pass

        logger.info(f"✅ Análisis completado: {len(unique_aggressors)} agresores, {len(unique_victims)} víctimas")

        if progress is not None:
            progress(1.0, "✅ Análisis completado!")

        return (
            len(unique_aggressors),
            len(unique_victims),
            processed_frame_idx,
            output_path,
            aggressor_frame_count,
            victim_frame_count,
            [],  # eventos
            first_agg_path,
            first_vic_path,
            first_both_path
        )

    except Exception as e:
        logger.error(f"❌ Error en process_video_full_analysis: {e}")
        import traceback
        logger.error(traceback.format_exc())
        return 0, 0, 0, None, 0, 0, [], None, None, None

# ==========================================================================================================
# EXTRACTOR DE FEATURES CORREGIDO
# ==========================================================================================================

def extract_lightweight_features(video_input, max_samples=10, yolo_model=None, progress=None):
    """Extrae features básicas para clasificación ML - VERSIÓN CORREGIDA"""
    try:
        if isinstance(video_input, str) and os.path.exists(video_input):
            video_path = video_input
        else:
            return np.zeros(6)  # Fallback

        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return np.zeros(6)

        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        sample_frames = min(max_samples, total_frames)

        # Features básicas pero reales
        person_counts = []
        motion_values = []

        for i in range(sample_frames):
            ret, frame = cap.read()
            if not ret:
                break

            frame_resized = cv2.resize(frame, (416, 416))

            # Contar personas
            results = yolo_model(frame_resized, verbose=False, classes=[0])
            person_count = len(results[0].boxes) if results else 0
            person_counts.append(person_count)

            # Movimiento simple (diferencia de frames)
            if i > 0:
                gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
                diff = cv2.absdiff(gray, prev_gray)
                motion_values.append(np.mean(diff))

            prev_frame = frame_resized.copy()

        cap.release()

        # Crear features
        features = np.array([
            np.mean(person_counts) if person_counts else 0,
            np.max(person_counts) if person_counts else 0,
            np.std(person_counts) if person_counts else 0,
            np.mean(motion_values) if motion_values else 0,
            np.max(motion_values) if motion_values else 0,
            total_frames / 1000.0  # Duración aproximada
        ])

        return features

    except Exception as e:
        logger.error(f"Error en extract_lightweight_features: {e}")
        return np.zeros(6)

# ==========================================================================================================
# FUNCIÓN PRINCIPAL MEJORADA
# ==========================================================================================================

def identificar_victima_y_agresor(
    video_input=None,
    youtube_url="",
    aggressor_threshold=2.8,
    victim_threshold=1.5,
    proximity_threshold=140,
    min_duration_frames=15,
    dist_threshold_px=85,
    speed_threshold=0.04,
    fist_distance_threshold=0.08,
    face_cover_threshold=0.10,
    crouch_threshold=0.25,
    movement_history=15,
    mostrar_interaccion="Ambos",
    progress=gr.Progress()
):
    """Función principal MEJORADA con detección REAL"""

    try:
        # Actualizar parámetros globales con tus valores MEJORADOS
        update_global_params({
            "AGGRESSOR_THRESHOLD": aggressor_threshold,
            "VICTIM_THRESHOLD": victim_threshold,
            "PROXIMITY_THRESHOLD": proximity_threshold,
            "MIN_DURATION_FRAMES": min_duration_frames,
            "dist_threshold_px": dist_threshold_px,
            "speed_threshold": speed_threshold,
            "FIST_DISTANCE_THRESHOLD": fist_distance_threshold,
            "FACE_COVER_THRESHOLD": face_cover_threshold,
            "CROUCH_THRESHOLD": crouch_threshold,
            "movement_history": movement_history,
        })

        video_path = None
        temp_file = None

        # Determinar fuente de video
        if youtube_url and youtube_url.strip():
            progress(0.1, "🌐 Descargando video de YouTube...")
            video_path = descargar_youtube(youtube_url, progress)
            temp_file = video_path
        elif video_input is not None:
            video_path = video_input
        else:
            raise gr.Error("❌ Por favor, sube un video o ingresa una URL de YouTube")

        progress(0.3, "🎯 Iniciando análisis avanzado del video...")

        # ✅ CORRECCIÓN: Procesar TODO el video
        (
            num_agresores,
            num_victimas,
            frames_procesados,
            video_procesado_path,
            frames_con_agresor,
            frames_con_victima,
            eventos,
            img_agresor,
            img_victima,
            img_ambos
        ) = process_video_full_analysis(
            video_path,
            output_dir='outputs_rwf',
            max_frames=None,  # ✅ Procesar TODO el video
            use_advanced=True,
            yolo_model=yolo_model,
            progress=progress,
            frame_skip=2  # Saltar un frame
        )

        progress(0.9, "📊 Generando resultados...")

        # Seleccionar imagen para mostrar
        img_mostrar = None
        mensaje_captura = ""

        if mostrar_interaccion == "Agresor" and img_agresor and os.path.exists(img_agresor):
            img_mostrar = img_agresor
            mensaje_captura = "🔴 Primera detección de AGRESOR"
        elif mostrar_interaccion == "Víctima" and img_victima and os.path.exists(img_victima):
            img_mostrar = img_victima
            mensaje_captura = "🔵 Primera detección de VÍCTIMA"
        elif mostrar_interaccion == "Ambos" and img_ambos and os.path.exists(img_ambos):
            img_mostrar = img_ambos
            mensaje_captura = "👥 Primera interacción AGRESOR + VÍCTIMA"
        else:
            mensaje_captura = "⚠ No se capturaron interacciones del tipo seleccionado"

        # ✅ REPORTE MEJORADO con información REAL
        violencia_detectada = (frames_con_agresor >= min_duration_frames and
                              frames_con_victima >= min_duration_frames)

        reporte = f"""
## 📊 Resultados del Análisis

### 👥 Detección de Personas
- **Agresores detectados:** {num_agresores}
- **Víctimas detectadas:** {num_victimas}
- **Frames procesados:** {frames_procesados}

### 🎯 Actividad Detectada
- **Frames con agresor:** {frames_con_agresor}
- **Frames con víctima:** {frames_con_victima}
- **Violencia prolongada:** {'✅ SÍ' if violencia_detectada else '❌ NO'}

### 📸 Capturas
- **{mensaje_captura}**

### ⚙️ Configuración Usada
- **Umbral agresor:** {aggressor_threshold}
- **Umbral víctima:** {victim_threshold}
- **Proximidad:** {proximity_threshold}px
- **Duración mínima:** {min_duration_frames} frames
"""

        # Clasificación ML MEJORADA (manejo de errores)
        if violence_classifier is not None:
            try:
                features = extract_lightweight_features(video_path, yolo_model=yolo_model)
                # ✅ CORRECCIÓN: Manejar el error de índice
                prediccion = violence_classifier.predict([features])[0]
                probabilidad = violence_classifier.predict_proba([features])[0]

                # Verificar la forma de las probabilidades
                if len(probabilidad) >= 2:
                    if prediccion == 1:
                        reporte += f"\n### 🔴 CLASIFICADOR ML: VIOLENCIA DETECTADA\n- **Confianza:** {probabilidad[1]:.1%}"
                    else:
                        reporte += f"\n### ✅ CLASIFICADOR ML: NO VIOLENCIA\n- **Confianza:** {probabilidad[0]:.1%}"
                elif len(probabilidad) == 1:
                    # Caso de clasificador con una sola clase
                    confianza = probabilidad[0]
                    if prediccion == 1:
                        reporte += f"\n### 🔴 CLASIFICADOR ML: VIOLENCIA DETECTADA\n- **Confianza:** {confianza:.1%}"
                    else:
                        reporte += f"\n### ✅ CLASIFICADOR ML: NO VIOLENCIA\n- **Confianza:** {confianza:.1%}"
                else:
                    reporte += f"\n### ⚠ CLASIFICADOR ML: Resultado - {prediccion}"

            except Exception as e:
                reporte += f"\n### ⚠ Clasificación ML no disponible\n- Error: {str(e)}"
        else:
            reporte += "\n### ℹ️ Clasificador ML no cargado"

        # Limpiar archivos temporales
        if temp_file and os.path.exists(temp_file):
            try:
                os.remove(temp_file)
            except:
                pass

        progress(1.0, "✅ ¡Análisis completado!")

        return video_procesado_path, img_mostrar, reporte

    except Exception as e:
        import traceback
        error_msg = f"Error durante el procesamiento: {str(e)}"
        logger.error(f"ERROR: {error_msg}")
        logger.error(traceback.format_exc())
        raise gr.Error(error_msg)

# ==========================================================================================================
# CARGAR MODELOS
# ==========================================================================================================

logger.info("\n" + "="*80)
logger.info("🚀 CARGANDO MODELOS")
logger.info("="*80)

logger.info("Cargando modelo YOLO...")
try:
    yolo_model = YOLO('yolov8n-pose.pt')
    logger.info("✓ Modelo YOLO cargado")
except Exception as e:
    logger.error(f"❌ Error cargando YOLO: {e}")
    yolo_model = None

# Intentar cargar clasificador de violencia
model_path = 'violence_classifier.pkl'
violence_classifier = None
if os.path.exists(model_path):
    try:
        violence_classifier = joblib.load(model_path)
        logger.info("✓ Clasificador de violencia cargado")
    except Exception as e:
        logger.error(f"⚠ Error cargando clasificador: {e}")
else:
    logger.warning("⚠ No se encontró violence_classifier.pkl - El sistema funcionará sin clasificación ML")

logger.info("="*80 + "\n")

# ==========================================================================================================
# INTERFAZ GRADIO
# ==========================================================================================================

def create_interface():
    """Crea y retorna la interfaz de Gradio"""

    with gr.Blocks(theme=gr.themes.Soft(), title="Identificador Víctima y Agresor") as demo:
        gr.Markdown("""
        # 🎯 Identificador Víctima y Agresor
        ### Sistema de Detección de Violencia con IA
        """)

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 📁 Fuente de Video")

                with gr.Tab("Subir Video"):
                    video_input = gr.Video(label="Seleccionar video", sources=["upload"])

                with gr.Tab("YouTube URL"):
                    youtube_url = gr.Textbox(
                        label="URL de YouTube",
                        placeholder="https://www.youtube.com/watch?v=...",
                        lines=1
                    )

                gr.Markdown("### ⚙️ Parámetros de Detección")

                with gr.Accordion("Parámetros Principales", open=True):
                    aggressor_threshold = gr.Slider(
                        minimum=0.5, maximum=5.0, value=2.8, step=0.1,
                        label="Umbral Agresor"
                    )
                    victim_threshold = gr.Slider(
                        minimum=0.5, maximum=5.0, value=1.5, step=0.1,
                        label="Umbral Víctima"
                    )
                    proximity_threshold = gr.Slider(
                        minimum=50, maximum=300, value=140, step=10,
                        label="Umbral de Proximidad (px)"
                    )
                    min_duration_frames = gr.Slider(
                        minimum=5, maximum=100, value=15, step=5,
                        label="Duración Mínima (frames)"
                    )

                with gr.Accordion("Parámetros Avanzados", open=False):
                    dist_threshold_px = gr.Slider(
                        minimum=20, maximum=200, value=85, step=5,
                        label="Distancia para Golpes (px)"
                    )
                    speed_threshold = gr.Slider(
                        minimum=0.01, maximum=0.2, value=0.04, step=0.01,
                        label="Umbral de Velocidad"
                    )
                    fist_distance_threshold = gr.Slider(
                        minimum=0.05, maximum=0.3, value=0.08, step=0.01,
                        label="Detección de Puños"
                    )
                    face_cover_threshold = gr.Slider(
                        minimum=0.05, maximum=0.3, value=0.10, step=0.01,
                        label="Protección Facial"
                    )
                    crouch_threshold = gr.Slider(
                        minimum=0.1, maximum=0.5, value=0.25, step=0.01,
                        label="Postura Agachada"
                    )
                    movement_history = gr.Slider(
                        minimum=5, maximum=50, value=15, step=5,
                        label="Historial de Movimiento"
                    )

                mostrar_interaccion = gr.Radio(
                    choices=["Ambos", "Agresor", "Víctima"],
                    value="Ambos",
                    label="📸 Mostrar en Resultados"
                )

                btn_analizar = gr.Button(
                    "🚀 Iniciar Análisis",
                    variant="primary",
                    size="lg"
                )

            with gr.Column(scale=2):
                gr.Markdown("### 📊 Resultados")

                with gr.Tab("Video Procesado"):
                    output_video = gr.Video(label="Video Analizado")

                with gr.Tab("Captura"):
                    output_screenshot = gr.Image(label="Primera Interacción Detectada")

                with gr.Tab("Reporte"):
                    output_text = gr.Markdown()

        # Conectar eventos
        btn_analizar.click(
            fn=identificar_victima_y_agresor,
            inputs=[
                video_input, youtube_url,
                aggressor_threshold, victim_threshold,
                proximity_threshold, min_duration_frames,
                dist_threshold_px, speed_threshold,
                fist_distance_threshold, face_cover_threshold,
                crouch_threshold, movement_history,
                mostrar_interaccion
            ],
            outputs=[output_video, output_screenshot, output_text]
        )

        gr.Markdown("""
        ---
        ### ℹ️ Instrucciones:
        1. Sube un video o ingresa una URL de YouTube
        2. Ajusta los parámetros según necesites
        3. Haz clic en 'Iniciar Análisis'
        4. Revisa los resultados en las pestañas

        **Nota:** El procesamiento puede tomar varios minutos dependiendo de la duración del video.
        """)

    return demo

# ==========================================================================================================
# LANZAR APLICACIÓN
# ==========================================================================================================

if __name__ == "__main__":
    logger.info("Iniciando aplicación...")

    # Crear interfaz
    demo = create_interface()

    # Detectar si estamos en Colab
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        logger.info("🏁 Entorno: Google Colab detectado")
        demo.launch(share=True, debug=False)
    else:
        logger.info("🏁 Entorno: Local/Jupyter detectado")
        import socket
        from contextlib import closing

        def find_free_port():
            with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
                s.bind(('', 0))
                s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                return s.getsockname()[1]

        free_port = find_free_port()
        logger.info(f"✅ Usando puerto: {free_port}")

        demo.launch(
            server_name="0.0.0.0",
            server_port=free_port,
            share=False,
            inbrowser=True,
            show_error=True
        )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1df560d07b1147bbb5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Mejorar entrenamiento de clasificador

In [ ]:
# ==========================================================================================================
# IDENTIFICADOR VÍCTIMA Y AGRESOR - SISTEMA DE DETECCIÓN DE VIOLENCIA FÍSICA
# ==========================================================================================================
# Fecha: 12 de noviembre de 2025
# País: Chile
# Versión: 1.2 - Sampleo de frames configurable globalmente
# ==========================================================================================================
#
# 🎯 CONFIGURACIÓN RÁPIDA DE SAMPLEO DE FRAMES:
#
# Para cambiar el número de frames procesados, modifica GLOBAL_PARAMS o usa:
#   configurar_sampleo_frames(training=40, prefilter=12, default=30)
#
# Presets recomendados:
#   ⚡ VELOCIDAD:    configurar_sampleo_frames(15, 8, 12)   → ~25 min entrenamiento
#   ⚖️ BALANCE:      configurar_sampleo_frames(30, 12, 25)  → ~50 min entrenamiento
#   🎯 PRECISIÓN:    configurar_sampleo_frames(50, 15, 40)  → ~85 min entrenamiento
#   🔬 MÁXIMO:       configurar_sampleo_frames(100, 20, 80) → ~170 min entrenamiento
#
# ==========================================================================================================

# 1. Instalación de dependencias
!pip install -q datasets huggingface_hub ultralytics tensorflow tensorflow_hub opencv-python tqdm scikit-learn matplotlib torch torchvision deep-sort-realtime yt_dlp joblib

import cv2
import numpy as np
import os
import tempfile
import json
from collections import defaultdict, deque
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import gradio as gr
import yt_dlp
import shutil
from pathlib import Path
from datasets import load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

print("✓ Dependencias instaladas correctamente")

# ==========================================================================================================
# CARGAR MODELO YOLO UNA VEZ
# ==========================================================================================================
print("Cargando modelo YOLO...")
try:
    yolo_model = YOLO('yolov8n-pose.pt')
    print("✓ Modelo YOLO cargado correctamente")
except Exception as e:
    print(f"❌ Error al cargar YOLO: {e}")
    yolo_model = None

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 31.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Dependencias instaladas correctamente
Cargando modelo YOLO...
✓ Modelo YOLO cargado correctamente


## Busqueda de hiperparametros (Queda pendiente límite RAM)

In [ ]:
# ============================================================================
# OPTIMIZACIÓN DE PARÁMETROS - RANDOM SEARCH DISTRIBUIDA
# ============================================================================

import itertools, random, json
from datetime import datetime
import pandas as pd

def random_search_parameter_optimization(dataset, machine_id=1, total_combinations=25, seed_base=42):
    """
    Ejecuta una búsqueda aleatoria de parámetros en paralelo (dividida por machine_id).

    Args:
        dataset: Dataset de entrada (por ejemplo, dataset['train'])
        machine_id: 1 o 2 (para separar combinaciones entre PCs)
        total_combinations: cantidad de configuraciones por máquina
        seed_base: base para la semilla aleatoria
    """

    # ------------------------------
    # 1. Definir rangos de parámetros
    # ------------------------------
    param_space = {
        "AGGRESSOR_THRESHOLD": (2.5, 5.0),
        "VICTIM_THRESHOLD": (1.5, 4.0),
        "PROXIMITY_THRESHOLD": (100, 200),
        "disappearance_threshold": (10, 40),
        "movement_history": (10, 25),
        "speed_threshold": (0.03, 0.1),
        "dist_threshold_px": (60, 120),
    }

    # ------------------------------
    # 2. Generar combinaciones aleatorias
    # ------------------------------
    seed = seed_base + machine_id
    random.seed(seed)
    param_combinations = []

    for _ in range(total_combinations):
        combo = {k: round(random.uniform(v[0], v[1]), 3) for k, v in param_space.items()}
        param_combinations.append(combo)

    # ------------------------------
    # 3. Ejecutar pruebas
    # ------------------------------
    results = []

    for idx, params in enumerate(param_combinations, start=1):
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] "
              f"Machine {machine_id} - Test {idx}/{total_combinations}")
        print("Params:", params)

        # Ejecutar con parámetros actuales
        try:
            aggressors, victims, frames, _ = process_video_rwf(
                random.choice(dataset['train']),  # usa un video aleatorio para test rápido
                max_frames=100,
                disappearance_threshold=int(params["disappearance_threshold"]),
                movement_history=int(params["movement_history"]),
                use_advanced=True
            )

            # Calcular métricas
            has_interaction = aggressors > 0 and victims > 0
            detection_density = (aggressors + victims) / max(frames, 1)
            precision, recall, f1, _ = precision_recall_fscore_support(
                [1 if has_interaction else 0], [1 if detection_density > 0.01 else 0],
                average='binary', pos_label=1, zero_division=0
            )

            results.append({
                **params,
                "precision": precision,
                "recall": recall,
                "f1": f1,
                "detection_density": detection_density,
                "has_interaction": has_interaction,
            })

        except Exception as e:
            print(f"⚠️ Error en combinación {idx}: {e}")
            results.append({**params, "precision": 0, "recall": 0, "f1": 0, "error": str(e)})

    # ------------------------------
    # 4. Guardar resultados
    # ------------------------------
    df_results = pd.DataFrame(results)
    out_path = f"param_opt_results_pc{machine_id}.csv"
    df_results.to_csv(out_path, index=False)
    print(f"\n✅ Resultados guardados en: {out_path}")

    # Mostrar top 5 por recall
    df_top = df_results.sort_values(by="recall", ascending=False).head(5)
    print("\n=== TOP 5 COMBINACIONES POR RECALL ===")
    print(df_top[["recall", "f1", "precision"] + list(param_space.keys())])

    return df_results

In [ ]:
# CAMILAAAAAA
results_pc1 = random_search_parameter_optimization(machine_id=1, total_combinations=25)

📂 Cargando dataset local desde 'rwf2000_cached'...
⚠️ No se pudo cargar desde 'rwf2000_cached', se descargará nuevamente. Error: No such files: '/content/rwf2000_cached/train/dataset_info.json', nor '/content/rwf2000_cached/train/state.json' found. Expected to load a `Dataset` object but provided path is not a `Dataset`.
⬇️ Descargando RWF-2000 desde Hugging Face...


Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

✅ Dataset descargado. Guardando copia local...


Saving the dataset (0/26 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

KeyboardInterrupt: 